In [8]:
from .header import get_header, get_logo, get_menu, Header
from .table import make_dash_table
from .printButton import print_button
from .functions import formatter_currency, formatter_currency_with_cents, formatter_percent, formatter_percent_2_digits, formatter_number
from .functions import update_first_datatable, update_first_download, update_second_datatable, update_graph

ModuleNotFoundError: No module named '__main__.header'; '__main__' is not a package

In [7]:
import dash_html_components as html
import dash_core_components as dcc

def Header():
    return html.Div([
        get_logo(),
        get_header(),
        html.Br([]),
        get_menu()
    ])

def get_logo():
    logo = html.Div([

        html.Div([
            html.Img(src='https://i.pinimg.com/564x/4a/bc/38/4abc38758eba60d6712bd86dd1542697.jpg', height='101', width='141')
        ], className="ten columns padded"),

        # html.Div([
        #     dcc.Link('Full View   ', href='/cc-travel-report/full-view')
        # ], className="two columns page-view no-print")

    ], className="row gs-header")
    return logo


def get_header():
    header = html.Div([

        html.Div([
            html.H5(
                'House Stark Performance Marketing Report')
        ], className="twelve columns padded")

    ], className="row gs-header gs-text-header")
    return header


def get_menu():
    menu = html.Div([

        dcc.Link('Overview - Birst   ', href='/cc-travel-report/overview-birst/', className="tab first"),

        dcc.Link('Overview - GA   ', href='/cc-travel-report/overview-ga/', className="tab"),

        dcc.Link('Paid Search   ', href='/cc-travel-report/paid-search/', className="tab"),

        dcc.Link('Display   ', href='/cc-travel-report/display/', className="tab"),

        dcc.Link('Publishing   ', href='/cc-travel-report/publishing/', className="tab"),

        dcc.Link('Metasearch and Travel Ads   ', href='/cc-travel-report/metasearch-and-travel-ads/', className="tab"),

    ], className="row ")
    return menu

In [9]:
import dash_html_components as html

def make_dash_table(df):
    ''' Return a dash definition of an HTML table for a Pandas dataframe '''
    table = []
    for index, row in df.iterrows():
        html_row = []
        for i in range(len(row)):
            html_row.append(html.Td([row[i]]))
        table.append(html.Tr(html_row))
    return table

In [10]:
import dash_html_components as html

def print_button():
    printButton = html.A(['Print PDF'],className="button no-print print",style={'position': "absolute", 'top': '-40', 'right': '0'})
    return printButton

In [15]:
from datetime import datetime as dt
from datetime import date, timedelta
from datetime import datetime
import plotly.graph_objs as go
from plotly import tools
import numpy as np
import pandas as pd

pd.options.mode.chained_assignment = None

# Read in Travel Report Data
df = pd.read_csv(r'C:\Users\mebra.DESKTOP-L12LJA6\Thinkful Works\PythonThinkful\performance_analytics_cost_and_ga_metrics.csv')

df.rename(columns={
 'Travel Product': 'Placement type', 
  'Spend - This Year': 'Spend TY', 
  'Spend - Last Year': 'Spend LY', 
  'Sessions - This Year': 'Sessions - TY',
  'Sessions - Last Year': 'Sessions - LY',
  'Bookings - This Year': 'Bookings - TY',
  'Bookings - Last Year': 'Bookings - LY',
  'Revenue - This Year': 'Revenue - TY',
  'Revenue - Last Year': 'Revenue - LY',
  }, inplace=True)


df['Date'] = pd.to_datetime(df['Date'])
current_year = df['Year'].max()
current_week = df[df['Year'] == current_year]['Week'].max()


now = datetime.now()
datestamp = now.strftime("%Y%m%d")

columns = ['Spend TY', 'Spend LY', 'Sessions - TY', 'Sessions - LY', 'Bookings - TY', 'Bookings - LY', 'Revenue - TY', 'Revenue - LY']


# Define Formatters
def formatter_currency(x):
	return "${:,.0f}".format(x) if x >= 0 else "(${:,.0f})".format(abs(x))

def formatter_currency_with_cents(x):
	return "${:,.2f}".format(x) if x >= 0 else "(${:,.2f})".format(abs(x))

def formatter_percent(x):
	return "{:,.1f}%".format(x) if x >= 0 else "({:,.1f}%)".format(abs(x))

def formatter_percent_2_digits(x):
	return "{:,.2f}%".format(x) if x >= 0 else "({:,.2f}%)".format(abs(x))

def formatter_number(x):
	return "{:,.0f}".format(x) if x >= 0 else "({:,.0f})".format(abs(x))


# First Data Table Update Function
def update_first_datatable(start_date, end_date, category, aggregation):
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%Y-%m-%d')
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%Y-%m-%d')
	days_selected = (end_date - start_date).days

	prior_start_date = start_date - timedelta(days_selected + 1)
	prior_start_date_string = datetime.strftime(prior_start_date, '%Y-%m-%d')
	prior_end_date = end_date - timedelta(days_selected + 1)
	prior_end_date_string = datetime.strftime(prior_end_date, '%Y-%m-%d')

	if aggregation == 'Placement type':
		df1 = df[(df['Category'] == category)].groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
	elif aggregation == 'GA Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'GA Category':'Placement type'}, inplace=True)
	elif aggregation == 'Birst Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'Birst Category':'Placement type'}, inplace=True)

	# Calculate Differences on-the-fly
	df_by_date_combined['Spend PoP (%)'] = np.nan
	df_by_date_combined['Spend YoY (%)'] = np.nan
	df_by_date_combined['Sessions PoP (%)'] = np.nan
	df_by_date_combined['Sessions YoY (%)'] = np.nan
	df_by_date_combined['Bookings PoP (%)'] = np.nan
	df_by_date_combined['Bookings YoY (%)'] = np.nan
	df_by_date_combined['Revenue PoP (%)'] = np.nan
	df_by_date_combined['Revenue YoY (%)'] = np.nan

	df_by_date_combined['Spend_PoP_abs_conditional'] = df_by_date_combined['Spend PoP (Abs)'] = ((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP']))
	# Formatter
	df_by_date_combined['Spend PoP (Abs)'] = df_by_date_combined['Spend PoP (Abs)'].apply(formatter_currency)

	df_by_date_combined['Spend_PoP_percent_conditional'] = df_by_date_combined['Spend PoP (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend - LP'] != 0),\
		(((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP'])/df_by_date_combined['Spend - LP']) * 100), df_by_date_combined['Spend PoP (%)'])
	# Formatter
	df_by_date_combined['Spend PoP (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend - LP'] != 0),\
		df_by_date_combined['Spend PoP (%)'].apply(formatter_percent), df_by_date_combined['Spend PoP (%)'])

	df_by_date_combined['Spend_YoY_percent_conditional'] = df_by_date_combined['Spend YoY (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend LY'] != 0),\
		((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend LY'])/df_by_date_combined['Spend LY']) * 100, df_by_date_combined['Spend YoY (%)'])
	# Formatter
	df_by_date_combined['Spend YoY (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend LY'] != 0),\
		df_by_date_combined['Spend YoY (%)'].apply(formatter_percent), df_by_date_combined['Spend YoY (%)'])


	df_by_date_combined['Sessions_PoP_percent_conditional'] = df_by_date_combined['Sessions PoP (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
		((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LP'])/df_by_date_combined['Sessions - LP']) * 100, df_by_date_combined['Sessions PoP (%)'])
	# Formatter
	df_by_date_combined['Sessions PoP (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
		df_by_date_combined['Sessions PoP (%)'].apply(formatter_percent), df_by_date_combined['Sessions PoP (%)'])

	df_by_date_combined['Sessions_YoY_percent_conditional'] = df_by_date_combined['Sessions YoY (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
		((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LY'])/df_by_date_combined['Sessions - LY']) * 100, df_by_date_combined['Sessions YoY (%)'])
	# Formatter
	df_by_date_combined['Sessions YoY (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
		df_by_date_combined['Sessions YoY (%)'].apply(formatter_percent), df_by_date_combined['Sessions YoY (%)'])


	df_by_date_combined['Bookings_PoP_abs_conditional'] = df_by_date_combined['Bookings PoP (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])
	# Formatter
	df_by_date_combined['Bookings PoP (Abs)'] = df_by_date_combined['Bookings PoP (Abs)'].apply(formatter_number)

	df_by_date_combined['Bookings_YoY_abs_conditional'] = df_by_date_combined['Bookings YoY (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])
	# Formatter
	df_by_date_combined['Bookings YoY (Abs)'] = df_by_date_combined['Bookings YoY (Abs)'].apply(formatter_number)

	df_by_date_combined['Bookings_PoP_percent_conditional'] = df_by_date_combined['Bookings PoP (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LP'] != 0),\
		(df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])/df_by_date_combined['Bookings - LP'] * 100, df_by_date_combined['Bookings PoP (%)'])
	# Formatter
	df_by_date_combined['Bookings PoP (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LP'] != 0),\
		df_by_date_combined['Bookings PoP (%)'].apply(formatter_percent), df_by_date_combined['Bookings PoP (%)'])

	df_by_date_combined['Bookings_YoY_percent_conditional'] = df_by_date_combined['Bookings YoY (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LY'] != 0),\
		(df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])/df_by_date_combined['Bookings - LY'] * 100, df_by_date_combined['Bookings YoY (%)'])
	# Formatter
	df_by_date_combined['Bookings YoY (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LY'] != 0),\
		df_by_date_combined['Bookings YoY (%)'].apply(formatter_percent), df_by_date_combined['Bookings YoY (%)'])


	df_by_date_combined['Revenue_PoP_abs_conditional'] = df_by_date_combined['Revenue PoP (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])
	# Formatter
	df_by_date_combined['Revenue PoP (Abs)'] = df_by_date_combined['Revenue PoP (Abs)'].apply(formatter_currency)

	df_by_date_combined['Revenue_YoY_abs_conditional'] = df_by_date_combined['Revenue YoY (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])
	# Formatter
	df_by_date_combined['Revenue YoY (Abs)'] = df_by_date_combined['Revenue YoY (Abs)'].apply(formatter_currency)

	df_by_date_combined['Revenue_PoP_percent_conditional'] = df_by_date_combined['Revenue PoP (%)'] = np.where((df_by_date_combined['Revenue - LP'] != 0) &  (df_by_date_combined['Revenue - LP'] != 0),\
		(df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])/df_by_date_combined['Revenue - LP'] * 100, df_by_date_combined['Revenue PoP (%)'])
	# Formatter
	df_by_date_combined['Revenue PoP (%)'] = np.where((df_by_date_combined['Revenue - LP'] != 0) &  (df_by_date_combined['Revenue - LP'] != 0),\
		df_by_date_combined['Revenue PoP (%)'].apply(formatter_percent), df_by_date_combined['Revenue PoP (%)'])

	df_by_date_combined['Revenue_YoY_percent_conditional'] = df_by_date_combined['Revenue YoY (%)'] = np.where((df_by_date_combined['Revenue - TY'] != 0) &  (df_by_date_combined['Revenue - LY'] != 0),\
		(df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])/df_by_date_combined['Revenue - LY'] * 100, df_by_date_combined['Revenue YoY (%)'])
	# Formatter
	df_by_date_combined['Revenue YoY (%)'] = np.where((df_by_date_combined['Revenue - TY'] != 0) &  (df_by_date_combined['Revenue - LY'] != 0),\
		df_by_date_combined['Revenue YoY (%)'].apply(formatter_percent), df_by_date_combined['Revenue YoY (%)'])

	# Format Numbers
	df_by_date_combined['Spend TY'] = df_by_date_combined['Spend TY'].apply(formatter_currency) 
	df_by_date_combined['Spend - LP'] = df_by_date_combined['Spend - LP'].apply(formatter_currency) 
	df_by_date_combined['Spend LY'] = df_by_date_combined['Spend LY'].apply(formatter_currency) 

	df_by_date_combined['Sessions - TY'] = df_by_date_combined['Sessions - TY'].apply(formatter_number)
	df_by_date_combined['Sessions - LP'] = df_by_date_combined['Sessions - LP'].apply(formatter_number)
	df_by_date_combined['Sessions - LY'] = df_by_date_combined['Sessions - LY'].apply(formatter_number)
	df_by_date_combined['Bookings - TY'] = df_by_date_combined['Bookings - TY'].apply(formatter_number)
	df_by_date_combined['Bookings - LP'] = df_by_date_combined['Bookings - LP'].apply(formatter_number)
	df_by_date_combined['Bookings - LY'] = df_by_date_combined['Bookings - LY'].apply(formatter_number)

	df_by_date_combined['Revenue - TY'] = df_by_date_combined['Revenue - TY'].apply(formatter_currency) 
	df_by_date_combined['Revenue - LP'] = df_by_date_combined['Revenue - LP'].apply(formatter_currency) 
	df_by_date_combined['Revenue - LY'] = df_by_date_combined['Revenue - LY'].apply(formatter_currency)
	# Rearrange the columns
	df_by_date_combined_dt = df_by_date_combined[[
		 'Placement type', 
		 'Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',
		 'Sessions - TY', 'Sessions - LP', 'Sessions PoP (%)', 'Sessions - LY', 'Sessions YoY (%)',
		 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 
		 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)', 
		 # 'Spend_PoP_percent_conditional', 
		 ]]

	data_df = df_by_date_combined.to_dict("rows")
	return data_df

# First Data Table Download Function
def update_first_download(start_date, end_date, category, aggregation):
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%Y-%m-%d')
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%Y-%m-%d')
	days_selected = (end_date - start_date).days

	prior_start_date = start_date - timedelta(days_selected + 1)
	prior_start_date_string = datetime.strftime(prior_start_date, '%Y-%m-%d')
	prior_end_date = end_date - timedelta(days_selected + 1)
	prior_end_date_string = datetime.strftime(prior_end_date, '%Y-%m-%d')

	if aggregation == 'Placement type':
		df1 = df[(df['Category'] == category)].groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
	elif aggregation == 'GA Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'GA Category':'Placement type'}, inplace=True)
	elif aggregation == 'Birst Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'Birst Category':'Placement type'}, inplace=True)

	# Calculate Differences on-the-fly
	df_by_date_combined['Spend PoP (%)'] = np.nan
	df_by_date_combined['Spend YoY (%)'] = np.nan
	df_by_date_combined['Sessions PoP (%)'] = np.nan
	df_by_date_combined['Sessions YoY (%)'] = np.nan
	df_by_date_combined['Bookings PoP (%)'] = np.nan
	df_by_date_combined['Bookings YoY (%)'] = np.nan
	df_by_date_combined['Revenue PoP (%)'] = np.nan
	df_by_date_combined['Revenue YoY (%)'] = np.nan

	df_by_date_combined['Spend PoP (Abs)'] = ((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP']))
	df_by_date_combined['Spend PoP (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend - LP'] != 0),\
		(((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP'])/df_by_date_combined['Spend - LP']) * 100), df_by_date_combined['Spend PoP (%)'])
	df_by_date_combined['Spend YoY (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend LY'] != 0),\
		((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend LY'])/df_by_date_combined['Spend LY']) * 100, df_by_date_combined['Spend YoY (%)'])

	df_by_date_combined['Sessions PoP (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
		((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LP'])/df_by_date_combined['Sessions - LP']) * 100, df_by_date_combined['Sessions PoP (%)'])
	df_by_date_combined['Sessions YoY (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
		((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LY'])/df_by_date_combined['Sessions - LY']) * 100, df_by_date_combined['Sessions YoY (%)'])

	df_by_date_combined['Bookings PoP (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])
	df_by_date_combined['Bookings YoY (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])

	df_by_date_combined['Bookings PoP (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LP'] != 0),\
		(df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])/df_by_date_combined['Bookings - LP'] * 100, df_by_date_combined['Bookings PoP (%)'])
	df_by_date_combined['Bookings YoY (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LY'] != 0),\
		(df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])/df_by_date_combined['Bookings - LY'] * 100, df_by_date_combined['Bookings YoY (%)'])

	df_by_date_combined['Revenue PoP (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])
	df_by_date_combined['Revenue YoY (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])

	df_by_date_combined['Revenue PoP (%)'] = np.where((df_by_date_combined['Revenue - LP'] != 0) &  (df_by_date_combined['Revenue - LP'] != 0),\
		(df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])/df_by_date_combined['Revenue - LP'] * 100, df_by_date_combined['Revenue PoP (%)'])
	df_by_date_combined['Revenue YoY (%)'] = np.where((df_by_date_combined['Revenue - TY'] != 0) &  (df_by_date_combined['Revenue - LY'] != 0),\
		(df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])/df_by_date_combined['Revenue - LY'] * 100, df_by_date_combined['Revenue YoY (%)'])

	# Calculate CPS, CR, CPA
	df_by_date_combined['CPS - TY'] = np.nan
	df_by_date_combined['CPS - LP'] = np.nan
	df_by_date_combined['CPS - LY'] = np.nan
	df_by_date_combined['CPS PoP (Abs)'] = np.nan
	df_by_date_combined['CPS YoY (Abs)'] = np.nan
	df_by_date_combined['CVR - TY'] = np.nan
	df_by_date_combined['CVR - LP'] = np.nan
	df_by_date_combined['CVR - LY'] = np.nan
	df_by_date_combined['CVR PoP (Abs)'] = np.nan
	df_by_date_combined['CVR YoY (Abs)'] = np.nan
	df_by_date_combined['CPA - TY'] = np.nan
	df_by_date_combined['CPA - LP'] = np.nan
	df_by_date_combined['CPA - LY'] = np.nan
	df_by_date_combined['CPA PoP (Abs)'] = np.nan
	df_by_date_combined['CPA YoY (Abs)'] = np.nan

	df_by_date_combined['CPS PoP (%)'] = np.nan
	df_by_date_combined['CPS YoY (%)'] = np.nan
	df_by_date_combined['CVR PoP (%)'] = np.nan
	df_by_date_combined['CVR YoY (%)'] = np.nan
	df_by_date_combined['CPA PoP (%)' ] = np.nan
	df_by_date_combined['CPA YoY (%)'] = np.nan

	df_by_date_combined['CPS - TY'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Sessions - TY'] != 0),\
									  (df_by_date_combined['Spend TY']/df_by_date_combined['Sessions - TY']), df_by_date_combined['CPS - TY'])

	df_by_date_combined['CPS - LP'] = np.where((df_by_date_combined['Spend - LP'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
									  (df_by_date_combined['Spend - LP']/df_by_date_combined['Sessions - LP']), df_by_date_combined['CPS - LP'])
	df_by_date_combined['CPS PoP (Abs)'] =  (df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LP'])

	df_by_date_combined['CPS PoP (%)'] =  np.where((df_by_date_combined['CPS - TY'] != 0) &  (df_by_date_combined['CPS - LP'] != 0),\
		((df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LP'])/df_by_date_combined['CPS - LP']), df_by_date_combined['CPS PoP (%)'])
					   
	df_by_date_combined['CPS - LY'] = np.where((df_by_date_combined['Spend LY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
									  (df_by_date_combined['Spend LY']/df_by_date_combined['Sessions - LY']), df_by_date_combined['CPS - LY'])
	df_by_date_combined['CPS YoY (Abs)'] =  (df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LY'])

	df_by_date_combined['CPS YoY (%)'] = np.where((df_by_date_combined['CPS - TY'] != 0) &  (df_by_date_combined['CPS - LY'] != 0),\
		((df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LY'])/df_by_date_combined['CPS - LY']), df_by_date_combined['CPS YoY (%)'] )

	df_by_date_combined['CVR - TY'] = np.where(((df_by_date_combined['Bookings - TY'] != 0) & (df_by_date_combined['Sessions - TY'] != 0)), \
									 (df_by_date_combined['Bookings - TY']/df_by_date_combined['Sessions - TY'] * 100), df_by_date_combined['CVR - TY'])
	df_by_date_combined['CVR - LP'] = np.where(((df_by_date_combined['Bookings - LP'] != 0) & (df_by_date_combined['Sessions - LP'] != 0)), \
							 (df_by_date_combined['Bookings - LP']/df_by_date_combined['Sessions - LP'] * 100), df_by_date_combined['CVR - LP'])
	df_by_date_combined['CVR PoP (Abs)'] =  np.where((df_by_date_combined['CVR - TY'].notnull() & df_by_date_combined['CVR - LP'].notnull()), \
							  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LP'])), df_by_date_combined['CVR PoP (Abs)'])

	df_by_date_combined['CVR PoP (%)'] =  np.where(((df_by_date_combined['CVR - TY'] != 0) & (df_by_date_combined['CVR - LP'] != 0)), \
		((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LP'])/df_by_date_combined['CVR - LP']), df_by_date_combined['CVR PoP (%)'])

	df_by_date_combined['CVR - LY'] = np.where(((df_by_date_combined['Bookings - LY'] != 0) & (df_by_date_combined['Sessions - LY'] != 0)), \
							 (df_by_date_combined['Bookings - LY']/df_by_date_combined['Sessions - LY'] * 100), df_by_date_combined['CVR - LY'])
	df_by_date_combined['CVR YoY (Abs)'] =  np.where((df_by_date_combined['CVR - TY'].notnull() & df_by_date_combined['CVR - LY'].notnull()), \
							  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LY'])), df_by_date_combined['CVR YoY (Abs)'])

	df_by_date_combined['CVR YoY (%)'] =  np.where(((df_by_date_combined['CVR - TY'] != 0) & (df_by_date_combined['CVR - LY'] != 0)), \
		((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LY'])/df_by_date_combined['CVR - LY']), df_by_date_combined['CVR YoY (%)'])

	df_by_date_combined['CPA - TY'] = np.where((df_by_date_combined['Spend TY'] != 0) & (df_by_date_combined['Bookings - TY'] != 0), \
									  (df_by_date_combined['Spend TY']/df_by_date_combined['Bookings - TY']), df_by_date_combined['CPA - TY'])
	df_by_date_combined['CPA - LP'] = np.where((df_by_date_combined['Spend - LP'] != 0) & (df_by_date_combined['Bookings - LP'] != 0), \
									  (df_by_date_combined['Spend - LP']/df_by_date_combined['Bookings - LP']), df_by_date_combined['CPA - LP'])
	df_by_date_combined['CPA PoP (Abs)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LP'] != 0), \
									  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LP']), df_by_date_combined['CPA PoP (Abs)'])

	df_by_date_combined['CPA PoP (%)' ] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LP'] != 0), \
		((df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LP'])/df_by_date_combined['CPA - LP']), df_by_date_combined['CPA PoP (%)' ] )


	df_by_date_combined['CPA - LY'] = np.where((df_by_date_combined['Spend LY'] != 0) & (df_by_date_combined['Bookings - LY'] != 0), \
									  (df_by_date_combined['Spend LY']/df_by_date_combined['Bookings - LY']), df_by_date_combined['CPA - LY'])
	df_by_date_combined['CPA YoY (Abs)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LY'] != 0), \
									  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LY']), df_by_date_combined['CPA YoY (Abs)'])

	df_by_date_combined['CPA YoY (%)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LY'] != 0), \
		(df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LY'])/df_by_date_combined['CPA - LY'], df_by_date_combined['CPA YoY (%)'])

	df_by_date_combined['TY Start Date'] = start_date_string
	df_by_date_combined['TY End Date'] = end_date_string
	df_by_date_combined['LP Start Date'] = prior_start_date_string
	df_by_date_combined['LP End Date'] = prior_end_date_string

	last_years_start_date = start_date - timedelta(364)
	last_years_start_date_string = datetime.strftime(last_years_start_date, '%Y-%m-%d')
	last_years_end_date = end_date - timedelta(364)
	last_years_end_date_string = datetime.strftime(last_years_end_date, '%Y-%m-%d')

	df_by_date_combined['LY Start Date'] = last_years_start_date_string
	df_by_date_combined['LY End Date'] = last_years_end_date_string

	# Rearrange the columns
	df_by_date_combined_dt = df_by_date_combined[[
		 'Placement type', 'TY Start Date', 'TY End Date', 'LP Start Date', 'LP End Date', 'LY Start Date', 'LY End Date',
		 'Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',
		 'Sessions - TY', 'Sessions - LP', 'Sessions PoP (%)', 'Sessions - LY', 'Sessions YoY (%)',
		 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 
		 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',  
		 'CPS - TY', 
		 'CPS - LP', 'CPS PoP (Abs)', 'CPS PoP (%)', 
		 'CPS - LY',  'CPS YoY (Abs)', 'CPS YoY (%)', 
		 'CVR - TY', 
		 'CVR - LP', 'CVR PoP (Abs)', 'CVR PoP (%)', 
		 'CVR - LY', 'CVR YoY (Abs)', 'CVR YoY (%)', 
		 'CPA - TY', 
		 'CPA - LP', 'CPA PoP (Abs)', 'CPA PoP (%)', 
		 'CPA - LY', 'CPA YoY (Abs)', 'CPA YoY (%)'
		 ]]


	download_df_1 = df_by_date_combined_dt
	return download_df_1

# Second Data Table Update Function
def update_second_datatable(start_date, end_date, category, aggregation):
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%Y-%m-%d')
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%Y-%m-%d')
	days_selected = (end_date - start_date).days

	prior_start_date = start_date - timedelta(days_selected + 1)
	prior_start_date_string = datetime.strftime(prior_start_date, '%Y-%m-%d')
	prior_end_date = end_date - timedelta(days_selected + 1)
	prior_end_date_string = datetime.strftime(prior_end_date, '%Y-%m-%d')

	if aggregation == 'Placement type':
		df1 = df[(df['Category'] == category)].groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
	elif aggregation == 'GA Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'GA Category':'Placement type'}, inplace=True)
	elif aggregation == 'Birst Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'Birst Category':'Placement type'}, inplace=True)

	# Calculate Differences on-the-fly
	# Calculate Percentage Changes
	df_by_date_combined['Spend PoP (Abs)'] = ((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP'])/df_by_date_combined['Spend - LP']) * 100
	df_by_date_combined['Spend PoP (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}%".format(x['Spend PoP (Abs)']), axis=1)
	df_by_date_combined['Spend YoY (%)'] = ((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend LY'])/df_by_date_combined['Spend LY']) * 100
	df_by_date_combined['Spend YoY (%)'] = df_by_date_combined.apply(lambda x: "{:,.0f}%".format(x['Spend YoY (%)']), axis=1)

	df_by_date_combined['Sessions PoP (%)'] = ((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LP'])/df_by_date_combined['Sessions - LP']) * 100
	df_by_date_combined['Sessions PoP (%)'] = df_by_date_combined.apply(lambda x: "{:,.0f}%".format(x['Sessions PoP (%)']), axis=1)
	df_by_date_combined['Sessions YoY (%)'] = ((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LY'])/df_by_date_combined['Sessions - LY']) * 100
	df_by_date_combined['Sessions YoY (%)'] = df_by_date_combined.apply(lambda x: "{:,.0f}%".format(x['Sessions YoY (%)']), axis=1)

	df_by_date_combined['Bookings PoP (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])
	df_by_date_combined['Bookings PoP (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}".format(x['Bookings PoP (Abs)']), axis=1)
	df_by_date_combined['Bookings YoY (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])
	df_by_date_combined['Bookings YoY (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}".format(x['Bookings YoY (Abs)']), axis=1)

	df_by_date_combined['Revenue PoP (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])
	df_by_date_combined['Revenue PoP (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}".format(x['Revenue PoP (Abs)']), axis=1)
	df_by_date_combined['Revenue YoY (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])
	df_by_date_combined['Revenue YoY (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}".format(x['Revenue YoY (Abs)']), axis=1)


	# Calculate CPS, CR, CPA
	df_by_date_combined['CPS - TY'] = np.nan
	df_by_date_combined['CPS - LP'] = np.nan
	df_by_date_combined['CPS - LY'] = np.nan
	df_by_date_combined['CPS PoP (Abs)'] = np.nan
	df_by_date_combined['CPS YoY (Abs)'] = np.nan
	df_by_date_combined['CVR - TY'] = np.nan
	df_by_date_combined['CVR - LP'] = np.nan
	df_by_date_combined['CVR - LY'] = np.nan
	df_by_date_combined['CVR PoP (Abs)'] = np.nan
	df_by_date_combined['CVR YoY (Abs)'] = np.nan
	df_by_date_combined['CPA - TY'] = np.nan
	df_by_date_combined['CPA - LP'] = np.nan
	df_by_date_combined['CPA - LY'] = np.nan
	df_by_date_combined['CPA PoP (Abs)'] = np.nan
	df_by_date_combined['CPA YoY (Abs)'] = np.nan

	df_by_date_combined['CPS - TY'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Sessions - TY'] != 0),\
									  (df_by_date_combined['Spend TY']/df_by_date_combined['Sessions - TY']), df_by_date_combined['CPS - TY'])

	df_by_date_combined['CPS - LP'] = np.where((df_by_date_combined['Spend - LP'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
									  (df_by_date_combined['Spend - LP']/df_by_date_combined['Sessions - LP']), df_by_date_combined['CPS - LP'])
	# df_by_date_combined['CPS_PoP_abs_conditional'] =  
	df_by_date_combined['CPS PoP (Abs)'] =  (df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LP'])
	df_by_date_combined['CPS_PoP_percent_conditional'] = df_by_date_combined['CPS PoP (%)'] = ((df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LP'])/df_by_date_combined['CPS - LP'] * 100)
					   
	df_by_date_combined['CPS - LY'] = np.where((df_by_date_combined['Spend LY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
									  (df_by_date_combined['Spend LY']/df_by_date_combined['Sessions - LY']), df_by_date_combined['CPS - LY'])
	df_by_date_combined['CPS_YoY_abs_conditional'] =  df_by_date_combined['CPS YoY (Abs)'] =  (df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LY'])
	df_by_date_combined['CPS_PoP_percent_conditional'] = df_by_date_combined['CPS YoY (%)'] = ((df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LY'])/df_by_date_combined['CPS - LY']) * 100

	df_by_date_combined['CVR - TY'] = np.where(((df_by_date_combined['Bookings - TY'] != 0) & (df_by_date_combined['Sessions - TY'] != 0)), \
									 (df_by_date_combined['Bookings - TY']/df_by_date_combined['Sessions - TY'] * 100), df_by_date_combined['CVR - TY'])
	df_by_date_combined['CVR - LP'] = np.where(((df_by_date_combined['Bookings - LP'] != 0) & (df_by_date_combined['Sessions - LP'] != 0)), \
							 (df_by_date_combined['Bookings - LP']/df_by_date_combined['Sessions - LP'] * 100), df_by_date_combined['CVR - LP'])

	df_by_date_combined['CVR_PoP_abs_conditional'] =  df_by_date_combined['CVR PoP (Abs)'] =  np.where((df_by_date_combined['CVR - TY'].notnull() & df_by_date_combined['CVR - LP'].notnull()), \
							  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LP'])), df_by_date_combined['CVR PoP (Abs)'])
	df_by_date_combined['CVR_PoP_percent_conditional'] =  df_by_date_combined['CVR PoP (%)'] =  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LP'])/df_by_date_combined['CVR - LP']) * 100
	df_by_date_combined['CVR - LY'] = np.where(((df_by_date_combined['Bookings - LY'] != 0) & (df_by_date_combined['Sessions - LY'] != 0)), \
							 (df_by_date_combined['Bookings - LY']/df_by_date_combined['Sessions - LY'] * 100), df_by_date_combined['CVR - LY'])
	df_by_date_combined['CVR_YoY_abs_conditional'] =  df_by_date_combined['CVR YoY (Abs)'] =  np.where((df_by_date_combined['CVR - TY'].notnull() & df_by_date_combined['CVR - LY'].notnull()), \
							  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LY'])), df_by_date_combined['CVR YoY (Abs)'])
	df_by_date_combined['CVR_YoY_percent_conditional'] =  df_by_date_combined['CVR YoY (%)'] =  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LY'])/df_by_date_combined['CVR - LY'] * 100)

	df_by_date_combined['CPA - TY'] = np.where((df_by_date_combined['Spend TY'] != 0) & (df_by_date_combined['Bookings - TY'] != 0), \
									  (df_by_date_combined['Spend TY']/df_by_date_combined['Bookings - TY']), df_by_date_combined['CPA - TY'])
	df_by_date_combined['CPA - LP'] = np.where((df_by_date_combined['Spend - LP'] != 0) & (df_by_date_combined['Bookings - LP'] != 0), \
									  (df_by_date_combined['Spend - LP']/df_by_date_combined['Bookings - LP']), df_by_date_combined['CPA - LP'])
	df_by_date_combined['CPA_PoP_abs_conditional'] =  df_by_date_combined['CPA PoP (Abs)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LP'] != 0), \
									  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LP']), df_by_date_combined['CPA PoP (Abs)'])

	df_by_date_combined['CPA_PoP_percent_conditional'] =  df_by_date_combined['CPA PoP (%)' ] =  ((df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LP'])/df_by_date_combined['CPA - LP'] * 100)


	df_by_date_combined['CPA - LY'] = np.where((df_by_date_combined['Spend LY'] != 0) & (df_by_date_combined['Bookings - LY'] != 0), \
									  (df_by_date_combined['Spend LY']/df_by_date_combined['Bookings - LY']), df_by_date_combined['CPA - LY'])
	df_by_date_combined['CPA_YoY_abs_conditional'] =  df_by_date_combined['CPA YoY (Abs)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LY'] != 0), \
									  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LY']), df_by_date_combined['CPA YoY (Abs)'])
	df_by_date_combined['CPA_YoY_percent_conditional'] =  df_by_date_combined['CPA YoY (%)'] =  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LY'])/df_by_date_combined['CPA - LY'] * 100

	df_by_date_combined['CPS_PoP_abs_conditional'] =  df_by_date_combined['CPS PoP (Abs)']

	#### REMEMBER FORMATTING MUST BE DONE AFTER MAKING CALCULATIONS
	df_by_date_combined['CPS - TY'] = np.where((df_by_date_combined['CPS - TY'].notnull()), \
		df_by_date_combined['CPS - TY'].apply(formatter_currency_with_cents), df_by_date_combined['CPS - TY'])

	df_by_date_combined['CPS - LP'] = np.where((df_by_date_combined['CPS - LP'].notnull()), \
		df_by_date_combined['CPS - LP'].apply(formatter_currency_with_cents), df_by_date_combined['CPS - LP'])

	df_by_date_combined['CPS - LY'] = np.where((df_by_date_combined['CPS - LY'].notnull()), \
		df_by_date_combined['CPS - LY'].apply(formatter_currency_with_cents), df_by_date_combined['CPS - LY'])

	df_by_date_combined['CPS PoP (Abs)'] = np.where((df_by_date_combined['CPS PoP (Abs)'].notnull()), \
		df_by_date_combined['CPS PoP (Abs)'].apply(formatter_currency_with_cents), df_by_date_combined['CPS PoP (Abs)'])

	df_by_date_combined['CPS YoY (Abs)'] = np.where((df_by_date_combined['CPS YoY (Abs)'].notnull()), \
		df_by_date_combined['CPS YoY (Abs)'].apply(formatter_currency_with_cents), df_by_date_combined['CPS YoY (Abs)'])

	df_by_date_combined['CPA - TY'] = np.where((df_by_date_combined['CPA - TY'].notnull()), \
		df_by_date_combined['CPA - TY'].apply(formatter_currency_with_cents), df_by_date_combined['CPA - TY'])

	df_by_date_combined['CPA - LP'] = np.where((df_by_date_combined['CPA - LP'].notnull()), \
		df_by_date_combined['CPA - LP'].apply(formatter_currency_with_cents), df_by_date_combined['CPA - LP'])

	df_by_date_combined['CPA - LY'] = np.where((df_by_date_combined['CPA - LY'].notnull()), \
		df_by_date_combined['CPA - LY'].apply(formatter_currency_with_cents), df_by_date_combined['CPA - LY'])

	df_by_date_combined['CPA PoP (Abs)'] = np.where((df_by_date_combined['CPA PoP (Abs)'].notnull()), \
	df_by_date_combined['CPA PoP (Abs)'].apply(formatter_currency_with_cents), df_by_date_combined['CPA PoP (Abs)'])

	df_by_date_combined['CPA YoY (Abs)'] = np.where((df_by_date_combined['CPA YoY (Abs)'].notnull()), \
		df_by_date_combined['CPA YoY (Abs)'].apply(formatter_currency_with_cents), df_by_date_combined['CPA YoY (Abs)'])

	df_by_date_combined['CPA PoP (%)'] = np.where((df_by_date_combined['CPA PoP (%)'].notnull()), \
		df_by_date_combined['CPA PoP (%)'].apply(formatter_percent), df_by_date_combined['CPA PoP (%)'])

	df_by_date_combined['CPA YoY (%)'] = np.where((df_by_date_combined['CPA YoY (%)'].notnull()), \
		df_by_date_combined['CPA YoY (%)'].apply(formatter_percent), df_by_date_combined['CPA YoY (%)'])

	df_by_date_combined['CPS PoP (%)'] = np.where((df_by_date_combined['CPS PoP (%)'].notnull()), \
		df_by_date_combined['CPS PoP (%)'].apply(formatter_percent), df_by_date_combined['CPS PoP (%)'])

	df_by_date_combined['CPS YoY (%)'] = np.where((df_by_date_combined['CPS YoY (%)'].notnull()), \
		df_by_date_combined['CPS YoY (%)'].apply(formatter_percent), df_by_date_combined['CPS YoY (%)'])

	df_by_date_combined['CVR PoP (%)'] = np.where((df_by_date_combined['CVR PoP (%)'].notnull()), \
		df_by_date_combined['CVR PoP (%)'].apply(formatter_percent), df_by_date_combined['CVR PoP (%)'])

	df_by_date_combined['CVR YoY (%)'] = np.where((df_by_date_combined['CVR YoY (%)'].notnull()), \
		df_by_date_combined['CVR YoY (%)'].apply(formatter_percent), df_by_date_combined['CVR YoY (%)'])

	df_by_date_combined['CVR - TY'] = np.where((df_by_date_combined['CVR - TY'].notnull()), \
		df_by_date_combined['CVR - TY'].apply(formatter_percent_2_digits), df_by_date_combined['CVR - TY'])

	df_by_date_combined['CVR - LP'] = np.where((df_by_date_combined['CVR - LP'].notnull()), \
		df_by_date_combined['CVR - LP'].apply(formatter_percent_2_digits), df_by_date_combined['CVR - LP'])

	df_by_date_combined['CVR - LY'] = np.where((df_by_date_combined['CVR - LY'].notnull()), \
		df_by_date_combined['CVR - LY'].apply(formatter_percent_2_digits), df_by_date_combined['CVR - LY'])

	df_by_date_combined['CVR PoP (Abs)'] = np.where((df_by_date_combined['CVR PoP (Abs)'].notnull()), \
		df_by_date_combined['CVR PoP (Abs)'].apply(formatter_percent_2_digits), df_by_date_combined['CVR PoP (Abs)'])

	df_by_date_combined['CVR YoY (Abs)'] = np.where((df_by_date_combined['CVR YoY (Abs)'].notnull()), \
		df_by_date_combined['CVR YoY (Abs)'].apply(formatter_percent_2_digits), df_by_date_combined['CVR YoY (Abs)'])


	# Rearrange the columns
	df_by_date_combined = df_by_date_combined[[
		'Placement type', 
		'CPS - TY', 
		'CPS - LP', 'CPS PoP (Abs)', 'CPS PoP (%)', 
		'CPS - LY',  'CPS YoY (Abs)', 'CPS YoY (%)', 
		'CVR - TY', 
		'CVR - LP', 'CVR PoP (Abs)', 'CVR PoP (%)', 
		'CVR - LY', 'CVR YoY (Abs)', 'CVR YoY (%)', 
		'CPA - TY', 
		'CPA - LP', 'CPA PoP (Abs)', 'CPA PoP (%)', 
		'CPA - LY', 'CPA YoY (Abs)', 'CPA YoY (%)',
		'CPS_PoP_abs_conditional', 'CPS_PoP_percent_conditional', 'CPS_YoY_abs_conditional', 'CPS_PoP_percent_conditional', 
		'CVR_PoP_abs_conditional', 'CVR_PoP_percent_conditional', 'CVR_YoY_abs_conditional', 'CVR_YoY_percent_conditional',
		'CPA_PoP_abs_conditional', 'CPA_PoP_percent_conditional', 'CPA_YoY_abs_conditional', 'CPA_YoY_percent_conditional'
		]]

	data_df = df_by_date_combined.to_dict("rows")
	return data_df

######################## FOR GRAPHS  ########################

def update_graph(filtered_df, end_date):
    if end_date is not None:
        end_date = dt.strptime(end_date, '%Y-%m-%d')
        end_date_string = end_date.strftime('%Y-%m-%d')
    if end_date_string <= '2018-12-29':
        current_year = 2018
    else:
        current_year = 2019

    # Calulate YoY Differences
    filtered_df['Spend YoY (%)'] = ((filtered_df['Spend TY'] - filtered_df['Spend LY'])/filtered_df['Spend LY']) * 100
    filtered_df['Sessions YoY (%)'] = ((filtered_df['Sessions - TY'] - filtered_df['Sessions - LY'])/filtered_df['Sessions - LY']) * 100
    filtered_df['Bookings - % - PY'] = ((filtered_df['Bookings - TY'] - filtered_df['Bookings - LY'])/filtered_df['Bookings - LY']) * 100
    filtered_df['Revenue - % - PY'] = ((filtered_df['Revenue - TY'] - filtered_df['Revenue - LY'])/filtered_df['Revenue - LY']) * 100

    # Calculate CPS, CR, CPA
    filtered_df['CPS - TY'] = np.nan
    filtered_df['CPS - LY'] = np.nan
    filtered_df['% YoY_CPS'] = np.nan

    filtered_df['CVR - TY'] = np.nan
    filtered_df['CVR - LY'] = np.nan
    filtered_df['CVR YoY (Abs)'] = np.nan

    filtered_df['CPA - TY'] = np.nan
    filtered_df['CPA - LY'] = np.nan
    filtered_df['% YoY_CPA'] = np.nan

    filtered_df['CPS - TY'] = np.where((filtered_df['Spend TY'] != 0) &  (filtered_df['Sessions - TY'] != 0), (filtered_df['Spend TY']/filtered_df['Sessions - TY']), filtered_df['CPS - TY'])                      
    filtered_df['CPS - LY'] = np.where((filtered_df['Spend LY'] != 0) &  (filtered_df['Sessions - LY'] != 0), (filtered_df['Spend LY']/filtered_df['Sessions - LY']), filtered_df['CPS - LY'])
    filtered_df['% YoY_CPS'] =  np.where((filtered_df['CPS - TY'] != 0) &  (filtered_df['CPS - LY'] != 0), ((filtered_df['CPS - TY'] - filtered_df['CPS - LY'])/filtered_df['CPS - LY']), filtered_df['% YoY_CPS'])

    filtered_df['CVR - TY'] = np.where(((filtered_df['Bookings - TY'] != 0) & (filtered_df['Sessions - TY'] != 0)), (filtered_df['Bookings - TY']/filtered_df['Sessions - TY'] * 100), filtered_df['CVR - TY'])
    filtered_df['CVR - LY'] = np.where(((filtered_df['Bookings - LY'] != 0) & (filtered_df['Sessions - LY'] != 0)), (filtered_df['Bookings - LY']/filtered_df['Sessions - LY'] * 100), filtered_df['CVR - LY'])
    filtered_df['CVR YoY (Abs)'] =  np.where((filtered_df['CVR - TY'].notnull() & filtered_df['CVR - LY'].notnull()), ((filtered_df['CVR - TY'] - filtered_df['CVR - LY'])), filtered_df['CVR YoY (Abs)'])

    filtered_df['CPA - TY'] = np.where((filtered_df['Spend TY'] != 0) & (filtered_df['Bookings - TY'] != 0), (filtered_df['Spend TY']/filtered_df['Bookings - TY']), filtered_df['CPA - TY'])                     
    filtered_df['CPA - LY'] = np.where((filtered_df['Spend LY'] != 0) & (filtered_df['Bookings - LY'] != 0), (filtered_df['Spend LY']/filtered_df['Bookings - LY']), filtered_df['CPA - LY'])
    filtered_df['% YoY_CPA'] =  np.where((filtered_df['CPA - TY'] != 0) & (filtered_df['CPA - LY'] != 0), ((filtered_df['CPA - TY'] - filtered_df['CPA - LY'])/filtered_df['CPA - LY']) * 100, filtered_df['% YoY_CPA'])


    # Sessions Graphs 
    sessions_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Sessions - TY'],
      text='Sessions - TY'
    )
    sessions_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['Sessions - TY'],
      text='Sessions - LY'
    )
    sessions_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Sessions YoY (%)'],
      text='Sessions YoY (%)', opacity=0.6
    )
    # Spend Graphs
    spend_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Spend TY'],
      text='Spend TY'
    )
    spend_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['Spend TY'],
      text='Spend LY'
    )
    spend_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Spend YoY (%)'],
      text='Spend YoY (%)', opacity=0.6
    )
    # Bookings Graphs
    bookings_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Bookings - TY'],
      text='Bookings - TY'
    )
    bookings_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['Bookings - TY'],
      text='Bookings - LY'
    )
    bookings_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Bookings - % - PY'],
      text='Bookings - % - PY', opacity=0.6
    )
    cpa_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['CPA - TY'],
      text='CPA - TY'
    )
    cpa_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['CPA - TY'],
      text='CPA - LY'
    )
    cpa_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['% YoY_CPA'],
      text='% CPA - YoY', opacity=0.6
    )
    cps_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['CPS - TY'],
      text='CPS - TY'
    )
    cps_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['CPS - TY'],
      text='CPS - LY'
    )
    cps_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['% YoY_CPS'],
      text='% CPS - YoY', opacity=0.6
    )
    cr_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['CVR - TY'],
      text='CVR - TY'
    )
    cr_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['CVR - TY'],
      text='CVR - LY'
    )
    cr_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['CVR YoY (Abs)'],
      text='CVR YoY (Abs)', opacity=0.6
    )

    fig = tools.make_subplots(
      rows=6, 
      cols=1, 
      shared_xaxes=True,
      subplot_titles=(								# Be sure to have same number of titles as number of graphs
        'Sessions', 
        'Spend',
        'Bookings', 
        'Cost per Acquisition',
        'CPS',
        'Conversion Rate'
        ))

    fig.append_trace(sessions_ty, 1, 1)		# 0
    fig.append_trace(sessions_ly, 1, 1)		# 1
    fig.append_trace(sessions_yoy, 1, 1)	# 2
    fig.append_trace(spend_ty, 2, 1)		# 3
    fig.append_trace(spend_ly, 2, 1)		# 4
    fig.append_trace(spend_yoy, 2, 1)		# 5
    fig.append_trace(bookings_ty, 3, 1)		# 6
    fig.append_trace(bookings_ly, 3, 1)		# 7
    fig.append_trace(bookings_yoy, 3, 1)	# 8
    fig.append_trace(cpa_ty, 4, 1)			# 9
    fig.append_trace(cpa_ly, 4, 1)			# 10
    fig.append_trace(cpa_yoy, 4, 1)			# 11
    fig.append_trace(cps_ty, 5, 1)			# 12
    fig.append_trace(cps_ly, 5, 1)			# 13
    fig.append_trace(cps_yoy, 5, 1)			# 14
    fig.append_trace(cr_ty, 6, 1)			# 15
    fig.append_trace(cr_ly, 6, 1)			# 16
    fig.append_trace(cr_yoy, 6, 1)			# 17

    # integer index below is the index of the trace
    # yaxis indices below need to start from the number of total graphs + 1 since they are on right-side
    # overlaing and anchor axes correspond to the graph number

    fig['data'][2].update(yaxis='y7')
    fig['layout']['yaxis7'] = dict(overlaying='y1', anchor='x1', side='right', showgrid=False, title='% Change YoY')

    fig['data'][5].update(yaxis='y8')
    fig['layout']['yaxis8'] = dict(overlaying='y2', anchor='x2', side='right', showgrid=False, title='% Change YoY')

    fig['data'][8].update(yaxis='y9')
    fig['layout']['yaxis9'] = dict(overlaying='y3', anchor='x3', side='right', showgrid=False, title='% Change YoY')

    fig['data'][11].update(yaxis='y10')
    fig['layout']['yaxis10'] = dict(overlaying='y4', anchor='x4', side='right', showgrid=False, title='% Change YoY')

    fig['data'][14].update(yaxis='y11')
    fig['layout']['yaxis11'] = dict(overlaying='y5', anchor='x5', side='right', showgrid=False, title='% Change YoY')

    fig['data'][17].update(yaxis='y12')
    fig['layout']['yaxis12'] = dict(overlaying='y6', anchor='x6', side='right', showgrid=False, title='% Change YoY')

    fig['layout']['xaxis'].update(title='Week of the Year' + ' - ' + str(current_year))
    for i in fig['layout']['annotations']:
      i['font'] = dict(size=12,
        # color='#ff0000'
        )
    fig['layout'].update(
      height= 1500,
      # width=750, 
      showlegend=False, 
      xaxis=dict(
        # tickmode='linear',
        # ticks='outside',
        # tick0=1,
        dtick=5,
        ticklen=8,
        tickwidth=2,
        tickcolor='#000',
        showgrid=True,
        zeroline=True,
        # showline=True,
        # mirror='ticks',
        # gridcolor='#bdbdbd',
        gridwidth=2
    ),
      )
    updated_fig = fig
    return updated_fig

In [22]:
import dash


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, url_base_pathname='/cc-travel-report/paid-search/')
server = app.server
app.config.suppress_callback_exceptions = True

#import dash_auth

#VALID_USERNAME_PASSWORD_PAIRS = [
#    ['alg', 'mexicovacation']
# ]

#auth = dash_auth.BasicAuth(app, VALID_USERNAME_PASSWORD_PAIRS
 #)

In [3]:
from dash.dependencies import Input, Output
import plotly.graph_objs as go
from plotly import tools

from datetime import datetime as dt
from datetime import date, timedelta
from datetime import datetime

import numpy as np
import pandas as pd

import io
import xlsxwriter
import flask
from flask import send_file




# from app import app

import dash


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, url_base_pathname='/cc-travel-report/paid-search/')
server = app.server
app.config.suppress_callback_exceptions = True


# from components import formatter_currency, formatter_currency_with_cents, formatter_percent, formatter_percent_2_digits, formatter_number
#from components import update_first_datatable, update_first_download, update_second_datatable, update_grap#h

pd.options.mode.chained_assignment = None

# Read in Travel Report Data
df = pd.read_csv('data/performance_analytics_cost_and_ga_metrics.csv')

df.rename(columns={
 'Travel Product': 'Placement type', 
  'Spend - This Year': 'Spend TY', 
  'Spend - Last Year': 'Spend LY', 
  'Sessions - This Year': 'Sessions - TY',
  'Sessions - Last Year': 'Sessions - LY',
  'Bookings - This Year': 'Bookings - TY',
  'Bookings - Last Year': 'Bookings - LY',
  'Revenue - This Year': 'Revenue - TY',
  'Revenue - Last Year': 'Revenue - LY',
  }, inplace=True)

df['Date'] = pd.to_datetime(df['Date'])
current_year = df['Year'].max()
current_week = df[df['Year'] == current_year]['Week'].max()

now = datetime.now()
datestamp = now.strftime("%Y%m%d")

columns = ['Spend TY', 'Spend LY', 'Sessions - TY', 'Sessions - LY', 'Bookings - TY', 'Bookings - LY', 'Revenue - TY', 'Revenue - LY']


# Define Formatters
def formatter_currency(x):
	return "${:,.0f}".format(x) if x >= 0 else "(${:,.0f})".format(abs(x))

def formatter_currency_with_cents(x):
	return "${:,.2f}".format(x) if x >= 0 else "(${:,.2f})".format(abs(x))

def formatter_percent(x):
	return "{:,.1f}%".format(x) if x >= 0 else "({:,.1f}%)".format(abs(x))

def formatter_percent_2_digits(x):
	return "{:,.2f}%".format(x) if x >= 0 else "({:,.2f}%)".format(abs(x))

def formatter_number(x):
	return "{:,.0f}".format(x) if x >= 0 else "({:,.0f})".format(abs(x))


# First Data Table Update Function
def update_first_datatable(start_date, end_date, category, aggregation):
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%Y-%m-%d')
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%Y-%m-%d')
	days_selected = (end_date - start_date).days

	prior_start_date = start_date - timedelta(days_selected + 1)
	prior_start_date_string = datetime.strftime(prior_start_date, '%Y-%m-%d')
	prior_end_date = end_date - timedelta(days_selected + 1)
	prior_end_date_string = datetime.strftime(prior_end_date, '%Y-%m-%d')

	if aggregation == 'Placement type':
		df1 = df[(df['Category'] == category)].groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
	elif aggregation == 'GA Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'GA Category':'Placement type'}, inplace=True)
	elif aggregation == 'Birst Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'Birst Category':'Placement type'}, inplace=True)

	# Calculate Differences on-the-fly
	df_by_date_combined['Spend PoP (%)'] = np.nan
	df_by_date_combined['Spend YoY (%)'] = np.nan
	df_by_date_combined['Sessions PoP (%)'] = np.nan
	df_by_date_combined['Sessions YoY (%)'] = np.nan
	df_by_date_combined['Bookings PoP (%)'] = np.nan
	df_by_date_combined['Bookings YoY (%)'] = np.nan
	df_by_date_combined['Revenue PoP (%)'] = np.nan
	df_by_date_combined['Revenue YoY (%)'] = np.nan

	df_by_date_combined['Spend_PoP_abs_conditional'] = df_by_date_combined['Spend PoP (Abs)'] = ((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP']))
	# Formatter
	df_by_date_combined['Spend PoP (Abs)'] = df_by_date_combined['Spend PoP (Abs)'].apply(formatter_currency)

	df_by_date_combined['Spend_PoP_percent_conditional'] = df_by_date_combined['Spend PoP (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend - LP'] != 0),\
		(((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP'])/df_by_date_combined['Spend - LP']) * 100), df_by_date_combined['Spend PoP (%)'])
	# Formatter
	df_by_date_combined['Spend PoP (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend - LP'] != 0),\
		df_by_date_combined['Spend PoP (%)'].apply(formatter_percent), df_by_date_combined['Spend PoP (%)'])

	df_by_date_combined['Spend_YoY_percent_conditional'] = df_by_date_combined['Spend YoY (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend LY'] != 0),\
		((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend LY'])/df_by_date_combined['Spend LY']) * 100, df_by_date_combined['Spend YoY (%)'])
	# Formatter
	df_by_date_combined['Spend YoY (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend LY'] != 0),\
		df_by_date_combined['Spend YoY (%)'].apply(formatter_percent), df_by_date_combined['Spend YoY (%)'])


	df_by_date_combined['Sessions_PoP_percent_conditional'] = df_by_date_combined['Sessions PoP (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
		((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LP'])/df_by_date_combined['Sessions - LP']) * 100, df_by_date_combined['Sessions PoP (%)'])
	# Formatter
	df_by_date_combined['Sessions PoP (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
		df_by_date_combined['Sessions PoP (%)'].apply(formatter_percent), df_by_date_combined['Sessions PoP (%)'])

	df_by_date_combined['Sessions_YoY_percent_conditional'] = df_by_date_combined['Sessions YoY (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
		((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LY'])/df_by_date_combined['Sessions - LY']) * 100, df_by_date_combined['Sessions YoY (%)'])
	# Formatter
	df_by_date_combined['Sessions YoY (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
		df_by_date_combined['Sessions YoY (%)'].apply(formatter_percent), df_by_date_combined['Sessions YoY (%)'])


	df_by_date_combined['Bookings_PoP_abs_conditional'] = df_by_date_combined['Bookings PoP (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])
	# Formatter
	df_by_date_combined['Bookings PoP (Abs)'] = df_by_date_combined['Bookings PoP (Abs)'].apply(formatter_number)

	df_by_date_combined['Bookings_YoY_abs_conditional'] = df_by_date_combined['Bookings YoY (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])
	# Formatter
	df_by_date_combined['Bookings YoY (Abs)'] = df_by_date_combined['Bookings YoY (Abs)'].apply(formatter_number)

	df_by_date_combined['Bookings_PoP_percent_conditional'] = df_by_date_combined['Bookings PoP (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LP'] != 0),\
		(df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])/df_by_date_combined['Bookings - LP'] * 100, df_by_date_combined['Bookings PoP (%)'])
	# Formatter
	df_by_date_combined['Bookings PoP (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LP'] != 0),\
		df_by_date_combined['Bookings PoP (%)'].apply(formatter_percent), df_by_date_combined['Bookings PoP (%)'])

	df_by_date_combined['Bookings_YoY_percent_conditional'] = df_by_date_combined['Bookings YoY (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LY'] != 0),\
		(df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])/df_by_date_combined['Bookings - LY'] * 100, df_by_date_combined['Bookings YoY (%)'])
	# Formatter
	df_by_date_combined['Bookings YoY (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LY'] != 0),\
		df_by_date_combined['Bookings YoY (%)'].apply(formatter_percent), df_by_date_combined['Bookings YoY (%)'])


	df_by_date_combined['Revenue_PoP_abs_conditional'] = df_by_date_combined['Revenue PoP (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])
	# Formatter
	df_by_date_combined['Revenue PoP (Abs)'] = df_by_date_combined['Revenue PoP (Abs)'].apply(formatter_currency)

	df_by_date_combined['Revenue_YoY_abs_conditional'] = df_by_date_combined['Revenue YoY (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])
	# Formatter
	df_by_date_combined['Revenue YoY (Abs)'] = df_by_date_combined['Revenue YoY (Abs)'].apply(formatter_currency)

	df_by_date_combined['Revenue_PoP_percent_conditional'] = df_by_date_combined['Revenue PoP (%)'] = np.where((df_by_date_combined['Revenue - LP'] != 0) &  (df_by_date_combined['Revenue - LP'] != 0),\
		(df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])/df_by_date_combined['Revenue - LP'] * 100, df_by_date_combined['Revenue PoP (%)'])
	# Formatter
	df_by_date_combined['Revenue PoP (%)'] = np.where((df_by_date_combined['Revenue - LP'] != 0) &  (df_by_date_combined['Revenue - LP'] != 0),\
		df_by_date_combined['Revenue PoP (%)'].apply(formatter_percent), df_by_date_combined['Revenue PoP (%)'])

	df_by_date_combined['Revenue_YoY_percent_conditional'] = df_by_date_combined['Revenue YoY (%)'] = np.where((df_by_date_combined['Revenue - TY'] != 0) &  (df_by_date_combined['Revenue - LY'] != 0),\
		(df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])/df_by_date_combined['Revenue - LY'] * 100, df_by_date_combined['Revenue YoY (%)'])
	# Formatter
	df_by_date_combined['Revenue YoY (%)'] = np.where((df_by_date_combined['Revenue - TY'] != 0) &  (df_by_date_combined['Revenue - LY'] != 0),\
		df_by_date_combined['Revenue YoY (%)'].apply(formatter_percent), df_by_date_combined['Revenue YoY (%)'])

	# Format Numbers
	df_by_date_combined['Spend TY'] = df_by_date_combined['Spend TY'].apply(formatter_currency) 
	df_by_date_combined['Spend - LP'] = df_by_date_combined['Spend - LP'].apply(formatter_currency) 
	df_by_date_combined['Spend LY'] = df_by_date_combined['Spend LY'].apply(formatter_currency) 

	df_by_date_combined['Sessions - TY'] = df_by_date_combined['Sessions - TY'].apply(formatter_number)
	df_by_date_combined['Sessions - LP'] = df_by_date_combined['Sessions - LP'].apply(formatter_number)
	df_by_date_combined['Sessions - LY'] = df_by_date_combined['Sessions - LY'].apply(formatter_number)
	df_by_date_combined['Bookings - TY'] = df_by_date_combined['Bookings - TY'].apply(formatter_number)
	df_by_date_combined['Bookings - LP'] = df_by_date_combined['Bookings - LP'].apply(formatter_number)
	df_by_date_combined['Bookings - LY'] = df_by_date_combined['Bookings - LY'].apply(formatter_number)

	df_by_date_combined['Revenue - TY'] = df_by_date_combined['Revenue - TY'].apply(formatter_currency) 
	df_by_date_combined['Revenue - LP'] = df_by_date_combined['Revenue - LP'].apply(formatter_currency) 
	df_by_date_combined['Revenue - LY'] = df_by_date_combined['Revenue - LY'].apply(formatter_currency)
	# Rearrange the columns
	df_by_date_combined_dt = df_by_date_combined[[
		 'Placement type', 
		 'Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',
		 'Sessions - TY', 'Sessions - LP', 'Sessions PoP (%)', 'Sessions - LY', 'Sessions YoY (%)',
		 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 
		 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)', 
		 # 'Spend_PoP_percent_conditional', 
		 ]]

	data_df = df_by_date_combined.to_dict("rows")
	return data_df

# First Data Table Download Function
def update_first_download(start_date, end_date, category, aggregation):
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%Y-%m-%d')
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%Y-%m-%d')
	days_selected = (end_date - start_date).days

	prior_start_date = start_date - timedelta(days_selected + 1)
	prior_start_date_string = datetime.strftime(prior_start_date, '%Y-%m-%d')
	prior_end_date = end_date - timedelta(days_selected + 1)
	prior_end_date_string = datetime.strftime(prior_end_date, '%Y-%m-%d')

	if aggregation == 'Placement type':
		df1 = df[(df['Category'] == category)].groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
	elif aggregation == 'GA Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'GA Category':'Placement type'}, inplace=True)
	elif aggregation == 'Birst Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'Birst Category':'Placement type'}, inplace=True)

	# Calculate Differences on-the-fly
	df_by_date_combined['Spend PoP (%)'] = np.nan
	df_by_date_combined['Spend YoY (%)'] = np.nan
	df_by_date_combined['Sessions PoP (%)'] = np.nan
	df_by_date_combined['Sessions YoY (%)'] = np.nan
	df_by_date_combined['Bookings PoP (%)'] = np.nan
	df_by_date_combined['Bookings YoY (%)'] = np.nan
	df_by_date_combined['Revenue PoP (%)'] = np.nan
	df_by_date_combined['Revenue YoY (%)'] = np.nan

	df_by_date_combined['Spend PoP (Abs)'] = ((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP']))
	df_by_date_combined['Spend PoP (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend - LP'] != 0),\
		(((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP'])/df_by_date_combined['Spend - LP']) * 100), df_by_date_combined['Spend PoP (%)'])
	df_by_date_combined['Spend YoY (%)'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Spend LY'] != 0),\
		((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend LY'])/df_by_date_combined['Spend LY']) * 100, df_by_date_combined['Spend YoY (%)'])

	df_by_date_combined['Sessions PoP (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
		((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LP'])/df_by_date_combined['Sessions - LP']) * 100, df_by_date_combined['Sessions PoP (%)'])
	df_by_date_combined['Sessions YoY (%)'] = np.where((df_by_date_combined['Sessions - TY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
		((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LY'])/df_by_date_combined['Sessions - LY']) * 100, df_by_date_combined['Sessions YoY (%)'])

	df_by_date_combined['Bookings PoP (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])
	df_by_date_combined['Bookings YoY (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])

	df_by_date_combined['Bookings PoP (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LP'] != 0),\
		(df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])/df_by_date_combined['Bookings - LP'] * 100, df_by_date_combined['Bookings PoP (%)'])
	df_by_date_combined['Bookings YoY (%)'] = np.where((df_by_date_combined['Bookings - TY'] != 0) &  (df_by_date_combined['Bookings - LY'] != 0),\
		(df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])/df_by_date_combined['Bookings - LY'] * 100, df_by_date_combined['Bookings YoY (%)'])

	df_by_date_combined['Revenue PoP (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])
	df_by_date_combined['Revenue YoY (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])

	df_by_date_combined['Revenue PoP (%)'] = np.where((df_by_date_combined['Revenue - LP'] != 0) &  (df_by_date_combined['Revenue - LP'] != 0),\
		(df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])/df_by_date_combined['Revenue - LP'] * 100, df_by_date_combined['Revenue PoP (%)'])
	df_by_date_combined['Revenue YoY (%)'] = np.where((df_by_date_combined['Revenue - TY'] != 0) &  (df_by_date_combined['Revenue - LY'] != 0),\
		(df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])/df_by_date_combined['Revenue - LY'] * 100, df_by_date_combined['Revenue YoY (%)'])

	# Calculate CPS, CR, CPA
	df_by_date_combined['CPS - TY'] = np.nan
	df_by_date_combined['CPS - LP'] = np.nan
	df_by_date_combined['CPS - LY'] = np.nan
	df_by_date_combined['CPS PoP (Abs)'] = np.nan
	df_by_date_combined['CPS YoY (Abs)'] = np.nan
	df_by_date_combined['CVR - TY'] = np.nan
	df_by_date_combined['CVR - LP'] = np.nan
	df_by_date_combined['CVR - LY'] = np.nan
	df_by_date_combined['CVR PoP (Abs)'] = np.nan
	df_by_date_combined['CVR YoY (Abs)'] = np.nan
	df_by_date_combined['CPA - TY'] = np.nan
	df_by_date_combined['CPA - LP'] = np.nan
	df_by_date_combined['CPA - LY'] = np.nan
	df_by_date_combined['CPA PoP (Abs)'] = np.nan
	df_by_date_combined['CPA YoY (Abs)'] = np.nan

	df_by_date_combined['CPS PoP (%)'] = np.nan
	df_by_date_combined['CPS YoY (%)'] = np.nan
	df_by_date_combined['CVR PoP (%)'] = np.nan
	df_by_date_combined['CVR YoY (%)'] = np.nan
	df_by_date_combined['CPA PoP (%)' ] = np.nan
	df_by_date_combined['CPA YoY (%)'] = np.nan

	df_by_date_combined['CPS - TY'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Sessions - TY'] != 0),\
									  (df_by_date_combined['Spend TY']/df_by_date_combined['Sessions - TY']), df_by_date_combined['CPS - TY'])

	df_by_date_combined['CPS - LP'] = np.where((df_by_date_combined['Spend - LP'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
									  (df_by_date_combined['Spend - LP']/df_by_date_combined['Sessions - LP']), df_by_date_combined['CPS - LP'])
	df_by_date_combined['CPS PoP (Abs)'] =  (df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LP'])

	df_by_date_combined['CPS PoP (%)'] =  np.where((df_by_date_combined['CPS - TY'] != 0) &  (df_by_date_combined['CPS - LP'] != 0),\
		((df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LP'])/df_by_date_combined['CPS - LP']), df_by_date_combined['CPS PoP (%)'])
					   
	df_by_date_combined['CPS - LY'] = np.where((df_by_date_combined['Spend LY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
									  (df_by_date_combined['Spend LY']/df_by_date_combined['Sessions - LY']), df_by_date_combined['CPS - LY'])
	df_by_date_combined['CPS YoY (Abs)'] =  (df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LY'])

	df_by_date_combined['CPS YoY (%)'] = np.where((df_by_date_combined['CPS - TY'] != 0) &  (df_by_date_combined['CPS - LY'] != 0),\
		((df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LY'])/df_by_date_combined['CPS - LY']), df_by_date_combined['CPS YoY (%)'] )

	df_by_date_combined['CVR - TY'] = np.where(((df_by_date_combined['Bookings - TY'] != 0) & (df_by_date_combined['Sessions - TY'] != 0)), \
									 (df_by_date_combined['Bookings - TY']/df_by_date_combined['Sessions - TY'] * 100), df_by_date_combined['CVR - TY'])
	df_by_date_combined['CVR - LP'] = np.where(((df_by_date_combined['Bookings - LP'] != 0) & (df_by_date_combined['Sessions - LP'] != 0)), \
							 (df_by_date_combined['Bookings - LP']/df_by_date_combined['Sessions - LP'] * 100), df_by_date_combined['CVR - LP'])
	df_by_date_combined['CVR PoP (Abs)'] =  np.where((df_by_date_combined['CVR - TY'].notnull() & df_by_date_combined['CVR - LP'].notnull()), \
							  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LP'])), df_by_date_combined['CVR PoP (Abs)'])

	df_by_date_combined['CVR PoP (%)'] =  np.where(((df_by_date_combined['CVR - TY'] != 0) & (df_by_date_combined['CVR - LP'] != 0)), \
		((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LP'])/df_by_date_combined['CVR - LP']), df_by_date_combined['CVR PoP (%)'])

	df_by_date_combined['CVR - LY'] = np.where(((df_by_date_combined['Bookings - LY'] != 0) & (df_by_date_combined['Sessions - LY'] != 0)), \
							 (df_by_date_combined['Bookings - LY']/df_by_date_combined['Sessions - LY'] * 100), df_by_date_combined['CVR - LY'])
	df_by_date_combined['CVR YoY (Abs)'] =  np.where((df_by_date_combined['CVR - TY'].notnull() & df_by_date_combined['CVR - LY'].notnull()), \
							  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LY'])), df_by_date_combined['CVR YoY (Abs)'])

	df_by_date_combined['CVR YoY (%)'] =  np.where(((df_by_date_combined['CVR - TY'] != 0) & (df_by_date_combined['CVR - LY'] != 0)), \
		((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LY'])/df_by_date_combined['CVR - LY']), df_by_date_combined['CVR YoY (%)'])

	df_by_date_combined['CPA - TY'] = np.where((df_by_date_combined['Spend TY'] != 0) & (df_by_date_combined['Bookings - TY'] != 0), \
									  (df_by_date_combined['Spend TY']/df_by_date_combined['Bookings - TY']), df_by_date_combined['CPA - TY'])
	df_by_date_combined['CPA - LP'] = np.where((df_by_date_combined['Spend - LP'] != 0) & (df_by_date_combined['Bookings - LP'] != 0), \
									  (df_by_date_combined['Spend - LP']/df_by_date_combined['Bookings - LP']), df_by_date_combined['CPA - LP'])
	df_by_date_combined['CPA PoP (Abs)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LP'] != 0), \
									  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LP']), df_by_date_combined['CPA PoP (Abs)'])

	df_by_date_combined['CPA PoP (%)' ] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LP'] != 0), \
		((df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LP'])/df_by_date_combined['CPA - LP']), df_by_date_combined['CPA PoP (%)' ] )


	df_by_date_combined['CPA - LY'] = np.where((df_by_date_combined['Spend LY'] != 0) & (df_by_date_combined['Bookings - LY'] != 0), \
									  (df_by_date_combined['Spend LY']/df_by_date_combined['Bookings - LY']), df_by_date_combined['CPA - LY'])
	df_by_date_combined['CPA YoY (Abs)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LY'] != 0), \
									  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LY']), df_by_date_combined['CPA YoY (Abs)'])

	df_by_date_combined['CPA YoY (%)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LY'] != 0), \
		(df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LY'])/df_by_date_combined['CPA - LY'], df_by_date_combined['CPA YoY (%)'])

	df_by_date_combined['TY Start Date'] = start_date_string
	df_by_date_combined['TY End Date'] = end_date_string
	df_by_date_combined['LP Start Date'] = prior_start_date_string
	df_by_date_combined['LP End Date'] = prior_end_date_string

	last_years_start_date = start_date - timedelta(364)
	last_years_start_date_string = datetime.strftime(last_years_start_date, '%Y-%m-%d')
	last_years_end_date = end_date - timedelta(364)
	last_years_end_date_string = datetime.strftime(last_years_end_date, '%Y-%m-%d')

	df_by_date_combined['LY Start Date'] = last_years_start_date_string
	df_by_date_combined['LY End Date'] = last_years_end_date_string

	# Rearrange the columns
	df_by_date_combined_dt = df_by_date_combined[[
		 'Placement type', 'TY Start Date', 'TY End Date', 'LP Start Date', 'LP End Date', 'LY Start Date', 'LY End Date',
		 'Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',
		 'Sessions - TY', 'Sessions - LP', 'Sessions PoP (%)', 'Sessions - LY', 'Sessions YoY (%)',
		 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 
		 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',  
		 'CPS - TY', 
		 'CPS - LP', 'CPS PoP (Abs)', 'CPS PoP (%)', 
		 'CPS - LY',  'CPS YoY (Abs)', 'CPS YoY (%)', 
		 'CVR - TY', 
		 'CVR - LP', 'CVR PoP (Abs)', 'CVR PoP (%)', 
		 'CVR - LY', 'CVR YoY (Abs)', 'CVR YoY (%)', 
		 'CPA - TY', 
		 'CPA - LP', 'CPA PoP (Abs)', 'CPA PoP (%)', 
		 'CPA - LY', 'CPA YoY (Abs)', 'CPA YoY (%)'
		 ]]


	download_df_1 = df_by_date_combined_dt
	return download_df_1

# Second Data Table Update Function
def update_second_datatable(start_date, end_date, category, aggregation):
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%Y-%m-%d')
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%Y-%m-%d')
	days_selected = (end_date - start_date).days

	prior_start_date = start_date - timedelta(days_selected + 1)
	prior_start_date_string = datetime.strftime(prior_start_date, '%Y-%m-%d')
	prior_end_date = end_date - timedelta(days_selected + 1)
	prior_end_date_string = datetime.strftime(prior_end_date, '%Y-%m-%d')

	if aggregation == 'Placement type':
		df1 = df[(df['Category'] == category)].groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
	elif aggregation == 'GA Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'GA Category':'Placement type'}, inplace=True)
	elif aggregation == 'Birst Category':
		df1 = df.groupby(['Date', aggregation]).sum()[columns].reset_index()
		df_by_date = df1[(df1['Date'] >= start_date_string) & (df1['Date'] <= end_date_string)].groupby([aggregation]).sum()[columns].reset_index()
		df_by_date_prior = df1[(df1['Date'] >= prior_start_date_string) & (df1['Date'] <= prior_end_date_string)].groupby([aggregation]).sum()[['Spend TY', 'Sessions - TY', 'Bookings - TY', 'Revenue - TY']].reset_index()
		df_by_date_prior.rename(columns={'Spend TY' : 'Spend - LP', 'Sessions - TY' : 'Sessions - LP',  'Bookings - TY' : 'Bookings - LP','Revenue - TY' : 'Revenue - LP'}, inplace=True)
		df_by_date_combined =  pd.merge(df_by_date, df_by_date_prior, on=[aggregation])
		df_by_date_combined.rename(columns={'Birst Category':'Placement type'}, inplace=True)

	# Calculate Differences on-the-fly
	# Calculate Percentage Changes
	df_by_date_combined['Spend PoP (Abs)'] = ((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend - LP'])/df_by_date_combined['Spend - LP']) * 100
	df_by_date_combined['Spend PoP (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}%".format(x['Spend PoP (Abs)']), axis=1)
	df_by_date_combined['Spend YoY (%)'] = ((df_by_date_combined['Spend TY'] - df_by_date_combined['Spend LY'])/df_by_date_combined['Spend LY']) * 100
	df_by_date_combined['Spend YoY (%)'] = df_by_date_combined.apply(lambda x: "{:,.0f}%".format(x['Spend YoY (%)']), axis=1)

	df_by_date_combined['Sessions PoP (%)'] = ((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LP'])/df_by_date_combined['Sessions - LP']) * 100
	df_by_date_combined['Sessions PoP (%)'] = df_by_date_combined.apply(lambda x: "{:,.0f}%".format(x['Sessions PoP (%)']), axis=1)
	df_by_date_combined['Sessions YoY (%)'] = ((df_by_date_combined['Sessions - TY'] - df_by_date_combined['Sessions - LY'])/df_by_date_combined['Sessions - LY']) * 100
	df_by_date_combined['Sessions YoY (%)'] = df_by_date_combined.apply(lambda x: "{:,.0f}%".format(x['Sessions YoY (%)']), axis=1)

	df_by_date_combined['Bookings PoP (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LP'])
	df_by_date_combined['Bookings PoP (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}".format(x['Bookings PoP (Abs)']), axis=1)
	df_by_date_combined['Bookings YoY (Abs)'] = (df_by_date_combined['Bookings - TY'] - df_by_date_combined['Bookings - LY'])
	df_by_date_combined['Bookings YoY (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}".format(x['Bookings YoY (Abs)']), axis=1)

	df_by_date_combined['Revenue PoP (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LP'])
	df_by_date_combined['Revenue PoP (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}".format(x['Revenue PoP (Abs)']), axis=1)
	df_by_date_combined['Revenue YoY (Abs)'] = (df_by_date_combined['Revenue - TY'] - df_by_date_combined['Revenue - LY'])
	df_by_date_combined['Revenue YoY (Abs)'] = df_by_date_combined.apply(lambda x: "{:,.0f}".format(x['Revenue YoY (Abs)']), axis=1)


	# Calculate CPS, CR, CPA
	df_by_date_combined['CPS - TY'] = np.nan
	df_by_date_combined['CPS - LP'] = np.nan
	df_by_date_combined['CPS - LY'] = np.nan
	df_by_date_combined['CPS PoP (Abs)'] = np.nan
	df_by_date_combined['CPS YoY (Abs)'] = np.nan
	df_by_date_combined['CVR - TY'] = np.nan
	df_by_date_combined['CVR - LP'] = np.nan
	df_by_date_combined['CVR - LY'] = np.nan
	df_by_date_combined['CVR PoP (Abs)'] = np.nan
	df_by_date_combined['CVR YoY (Abs)'] = np.nan
	df_by_date_combined['CPA - TY'] = np.nan
	df_by_date_combined['CPA - LP'] = np.nan
	df_by_date_combined['CPA - LY'] = np.nan
	df_by_date_combined['CPA PoP (Abs)'] = np.nan
	df_by_date_combined['CPA YoY (Abs)'] = np.nan

	df_by_date_combined['CPS - TY'] = np.where((df_by_date_combined['Spend TY'] != 0) &  (df_by_date_combined['Sessions - TY'] != 0),\
									  (df_by_date_combined['Spend TY']/df_by_date_combined['Sessions - TY']), df_by_date_combined['CPS - TY'])

	df_by_date_combined['CPS - LP'] = np.where((df_by_date_combined['Spend - LP'] != 0) &  (df_by_date_combined['Sessions - LP'] != 0),\
									  (df_by_date_combined['Spend - LP']/df_by_date_combined['Sessions - LP']), df_by_date_combined['CPS - LP'])
	# df_by_date_combined['CPS_PoP_abs_conditional'] =  
	df_by_date_combined['CPS PoP (Abs)'] =  (df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LP'])
	df_by_date_combined['CPS_PoP_percent_conditional'] = df_by_date_combined['CPS PoP (%)'] = ((df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LP'])/df_by_date_combined['CPS - LP'] * 100)
					   
	df_by_date_combined['CPS - LY'] = np.where((df_by_date_combined['Spend LY'] != 0) &  (df_by_date_combined['Sessions - LY'] != 0),\
									  (df_by_date_combined['Spend LY']/df_by_date_combined['Sessions - LY']), df_by_date_combined['CPS - LY'])
	df_by_date_combined['CPS_YoY_abs_conditional'] =  df_by_date_combined['CPS YoY (Abs)'] =  (df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LY'])
	df_by_date_combined['CPS_PoP_percent_conditional'] = df_by_date_combined['CPS YoY (%)'] = ((df_by_date_combined['CPS - TY'] - df_by_date_combined['CPS - LY'])/df_by_date_combined['CPS - LY']) * 100

	df_by_date_combined['CVR - TY'] = np.where(((df_by_date_combined['Bookings - TY'] != 0) & (df_by_date_combined['Sessions - TY'] != 0)), \
									 (df_by_date_combined['Bookings - TY']/df_by_date_combined['Sessions - TY'] * 100), df_by_date_combined['CVR - TY'])
	df_by_date_combined['CVR - LP'] = np.where(((df_by_date_combined['Bookings - LP'] != 0) & (df_by_date_combined['Sessions - LP'] != 0)), \
							 (df_by_date_combined['Bookings - LP']/df_by_date_combined['Sessions - LP'] * 100), df_by_date_combined['CVR - LP'])

	df_by_date_combined['CVR_PoP_abs_conditional'] =  df_by_date_combined['CVR PoP (Abs)'] =  np.where((df_by_date_combined['CVR - TY'].notnull() & df_by_date_combined['CVR - LP'].notnull()), \
							  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LP'])), df_by_date_combined['CVR PoP (Abs)'])
	df_by_date_combined['CVR_PoP_percent_conditional'] =  df_by_date_combined['CVR PoP (%)'] =  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LP'])/df_by_date_combined['CVR - LP']) * 100
	df_by_date_combined['CVR - LY'] = np.where(((df_by_date_combined['Bookings - LY'] != 0) & (df_by_date_combined['Sessions - LY'] != 0)), \
							 (df_by_date_combined['Bookings - LY']/df_by_date_combined['Sessions - LY'] * 100), df_by_date_combined['CVR - LY'])
	df_by_date_combined['CVR_YoY_abs_conditional'] =  df_by_date_combined['CVR YoY (Abs)'] =  np.where((df_by_date_combined['CVR - TY'].notnull() & df_by_date_combined['CVR - LY'].notnull()), \
							  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LY'])), df_by_date_combined['CVR YoY (Abs)'])
	df_by_date_combined['CVR_YoY_percent_conditional'] =  df_by_date_combined['CVR YoY (%)'] =  ((df_by_date_combined['CVR - TY'] - df_by_date_combined['CVR - LY'])/df_by_date_combined['CVR - LY'] * 100)

	df_by_date_combined['CPA - TY'] = np.where((df_by_date_combined['Spend TY'] != 0) & (df_by_date_combined['Bookings - TY'] != 0), \
									  (df_by_date_combined['Spend TY']/df_by_date_combined['Bookings - TY']), df_by_date_combined['CPA - TY'])
	df_by_date_combined['CPA - LP'] = np.where((df_by_date_combined['Spend - LP'] != 0) & (df_by_date_combined['Bookings - LP'] != 0), \
									  (df_by_date_combined['Spend - LP']/df_by_date_combined['Bookings - LP']), df_by_date_combined['CPA - LP'])
	df_by_date_combined['CPA_PoP_abs_conditional'] =  df_by_date_combined['CPA PoP (Abs)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LP'] != 0), \
									  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LP']), df_by_date_combined['CPA PoP (Abs)'])

	df_by_date_combined['CPA_PoP_percent_conditional'] =  df_by_date_combined['CPA PoP (%)' ] =  ((df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LP'])/df_by_date_combined['CPA - LP'] * 100)


	df_by_date_combined['CPA - LY'] = np.where((df_by_date_combined['Spend LY'] != 0) & (df_by_date_combined['Bookings - LY'] != 0), \
									  (df_by_date_combined['Spend LY']/df_by_date_combined['Bookings - LY']), df_by_date_combined['CPA - LY'])
	df_by_date_combined['CPA_YoY_abs_conditional'] =  df_by_date_combined['CPA YoY (Abs)'] =  np.where((df_by_date_combined['CPA - TY'] != 0) & (df_by_date_combined['CPA - LY'] != 0), \
									  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LY']), df_by_date_combined['CPA YoY (Abs)'])
	df_by_date_combined['CPA_YoY_percent_conditional'] =  df_by_date_combined['CPA YoY (%)'] =  (df_by_date_combined['CPA - TY'] - df_by_date_combined['CPA - LY'])/df_by_date_combined['CPA - LY'] * 100

	df_by_date_combined['CPS_PoP_abs_conditional'] =  df_by_date_combined['CPS PoP (Abs)']

	#### REMEMBER FORMATTING MUST BE DONE AFTER MAKING CALCULATIONS
	df_by_date_combined['CPS - TY'] = np.where((df_by_date_combined['CPS - TY'].notnull()), \
		df_by_date_combined['CPS - TY'].apply(formatter_currency_with_cents), df_by_date_combined['CPS - TY'])

	df_by_date_combined['CPS - LP'] = np.where((df_by_date_combined['CPS - LP'].notnull()), \
		df_by_date_combined['CPS - LP'].apply(formatter_currency_with_cents), df_by_date_combined['CPS - LP'])

	df_by_date_combined['CPS - LY'] = np.where((df_by_date_combined['CPS - LY'].notnull()), \
		df_by_date_combined['CPS - LY'].apply(formatter_currency_with_cents), df_by_date_combined['CPS - LY'])

	df_by_date_combined['CPS PoP (Abs)'] = np.where((df_by_date_combined['CPS PoP (Abs)'].notnull()), \
		df_by_date_combined['CPS PoP (Abs)'].apply(formatter_currency_with_cents), df_by_date_combined['CPS PoP (Abs)'])

	df_by_date_combined['CPS YoY (Abs)'] = np.where((df_by_date_combined['CPS YoY (Abs)'].notnull()), \
		df_by_date_combined['CPS YoY (Abs)'].apply(formatter_currency_with_cents), df_by_date_combined['CPS YoY (Abs)'])

	df_by_date_combined['CPA - TY'] = np.where((df_by_date_combined['CPA - TY'].notnull()), \
		df_by_date_combined['CPA - TY'].apply(formatter_currency_with_cents), df_by_date_combined['CPA - TY'])

	df_by_date_combined['CPA - LP'] = np.where((df_by_date_combined['CPA - LP'].notnull()), \
		df_by_date_combined['CPA - LP'].apply(formatter_currency_with_cents), df_by_date_combined['CPA - LP'])

	df_by_date_combined['CPA - LY'] = np.where((df_by_date_combined['CPA - LY'].notnull()), \
		df_by_date_combined['CPA - LY'].apply(formatter_currency_with_cents), df_by_date_combined['CPA - LY'])

	df_by_date_combined['CPA PoP (Abs)'] = np.where((df_by_date_combined['CPA PoP (Abs)'].notnull()), \
	df_by_date_combined['CPA PoP (Abs)'].apply(formatter_currency_with_cents), df_by_date_combined['CPA PoP (Abs)'])

	df_by_date_combined['CPA YoY (Abs)'] = np.where((df_by_date_combined['CPA YoY (Abs)'].notnull()), \
		df_by_date_combined['CPA YoY (Abs)'].apply(formatter_currency_with_cents), df_by_date_combined['CPA YoY (Abs)'])

	df_by_date_combined['CPA PoP (%)'] = np.where((df_by_date_combined['CPA PoP (%)'].notnull()), \
		df_by_date_combined['CPA PoP (%)'].apply(formatter_percent), df_by_date_combined['CPA PoP (%)'])

	df_by_date_combined['CPA YoY (%)'] = np.where((df_by_date_combined['CPA YoY (%)'].notnull()), \
		df_by_date_combined['CPA YoY (%)'].apply(formatter_percent), df_by_date_combined['CPA YoY (%)'])

	df_by_date_combined['CPS PoP (%)'] = np.where((df_by_date_combined['CPS PoP (%)'].notnull()), \
		df_by_date_combined['CPS PoP (%)'].apply(formatter_percent), df_by_date_combined['CPS PoP (%)'])

	df_by_date_combined['CPS YoY (%)'] = np.where((df_by_date_combined['CPS YoY (%)'].notnull()), \
		df_by_date_combined['CPS YoY (%)'].apply(formatter_percent), df_by_date_combined['CPS YoY (%)'])

	df_by_date_combined['CVR PoP (%)'] = np.where((df_by_date_combined['CVR PoP (%)'].notnull()), \
		df_by_date_combined['CVR PoP (%)'].apply(formatter_percent), df_by_date_combined['CVR PoP (%)'])

	df_by_date_combined['CVR YoY (%)'] = np.where((df_by_date_combined['CVR YoY (%)'].notnull()), \
		df_by_date_combined['CVR YoY (%)'].apply(formatter_percent), df_by_date_combined['CVR YoY (%)'])

	df_by_date_combined['CVR - TY'] = np.where((df_by_date_combined['CVR - TY'].notnull()), \
		df_by_date_combined['CVR - TY'].apply(formatter_percent_2_digits), df_by_date_combined['CVR - TY'])

	df_by_date_combined['CVR - LP'] = np.where((df_by_date_combined['CVR - LP'].notnull()), \
		df_by_date_combined['CVR - LP'].apply(formatter_percent_2_digits), df_by_date_combined['CVR - LP'])

	df_by_date_combined['CVR - LY'] = np.where((df_by_date_combined['CVR - LY'].notnull()), \
		df_by_date_combined['CVR - LY'].apply(formatter_percent_2_digits), df_by_date_combined['CVR - LY'])

	df_by_date_combined['CVR PoP (Abs)'] = np.where((df_by_date_combined['CVR PoP (Abs)'].notnull()), \
		df_by_date_combined['CVR PoP (Abs)'].apply(formatter_percent_2_digits), df_by_date_combined['CVR PoP (Abs)'])

	df_by_date_combined['CVR YoY (Abs)'] = np.where((df_by_date_combined['CVR YoY (Abs)'].notnull()), \
		df_by_date_combined['CVR YoY (Abs)'].apply(formatter_percent_2_digits), df_by_date_combined['CVR YoY (Abs)'])


	# Rearrange the columns
	df_by_date_combined = df_by_date_combined[[
		'Placement type', 
		'CPS - TY', 
		'CPS - LP', 'CPS PoP (Abs)', 'CPS PoP (%)', 
		'CPS - LY',  'CPS YoY (Abs)', 'CPS YoY (%)', 
		'CVR - TY', 
		'CVR - LP', 'CVR PoP (Abs)', 'CVR PoP (%)', 
		'CVR - LY', 'CVR YoY (Abs)', 'CVR YoY (%)', 
		'CPA - TY', 
		'CPA - LP', 'CPA PoP (Abs)', 'CPA PoP (%)', 
		'CPA - LY', 'CPA YoY (Abs)', 'CPA YoY (%)',
		'CPS_PoP_abs_conditional', 'CPS_PoP_percent_conditional', 'CPS_YoY_abs_conditional', 'CPS_PoP_percent_conditional', 
		'CVR_PoP_abs_conditional', 'CVR_PoP_percent_conditional', 'CVR_YoY_abs_conditional', 'CVR_YoY_percent_conditional',
		'CPA_PoP_abs_conditional', 'CPA_PoP_percent_conditional', 'CPA_YoY_abs_conditional', 'CPA_YoY_percent_conditional'
		]]

	data_df = df_by_date_combined.to_dict("rows")
	return data_df

######################## FOR GRAPHS  ########################

def update_graph(filtered_df, end_date):
    if end_date is not None:
        end_date = dt.strptime(end_date, '%Y-%m-%d')
        end_date_string = end_date.strftime('%Y-%m-%d')
    if end_date_string <= '2018-12-29':
        current_year = 2018
    else:
        current_year = 2019

    # Calulate YoY Differences
    filtered_df['Spend YoY (%)'] = ((filtered_df['Spend TY'] - filtered_df['Spend LY'])/filtered_df['Spend LY']) * 100
    filtered_df['Sessions YoY (%)'] = ((filtered_df['Sessions - TY'] - filtered_df['Sessions - LY'])/filtered_df['Sessions - LY']) * 100
    filtered_df['Bookings - % - PY'] = ((filtered_df['Bookings - TY'] - filtered_df['Bookings - LY'])/filtered_df['Bookings - LY']) * 100
    filtered_df['Revenue - % - PY'] = ((filtered_df['Revenue - TY'] - filtered_df['Revenue - LY'])/filtered_df['Revenue - LY']) * 100

    # Calculate CPS, CR, CPA
    filtered_df['CPS - TY'] = np.nan
    filtered_df['CPS - LY'] = np.nan
    filtered_df['% YoY_CPS'] = np.nan

    filtered_df['CVR - TY'] = np.nan
    filtered_df['CVR - LY'] = np.nan
    filtered_df['CVR YoY (Abs)'] = np.nan

    filtered_df['CPA - TY'] = np.nan
    filtered_df['CPA - LY'] = np.nan
    filtered_df['% YoY_CPA'] = np.nan

    filtered_df['CPS - TY'] = np.where((filtered_df['Spend TY'] != 0) &  (filtered_df['Sessions - TY'] != 0), (filtered_df['Spend TY']/filtered_df['Sessions - TY']), filtered_df['CPS - TY'])                      
    filtered_df['CPS - LY'] = np.where((filtered_df['Spend LY'] != 0) &  (filtered_df['Sessions - LY'] != 0), (filtered_df['Spend LY']/filtered_df['Sessions - LY']), filtered_df['CPS - LY'])
    filtered_df['% YoY_CPS'] =  np.where((filtered_df['CPS - TY'] != 0) &  (filtered_df['CPS - LY'] != 0), ((filtered_df['CPS - TY'] - filtered_df['CPS - LY'])/filtered_df['CPS - LY']), filtered_df['% YoY_CPS'])

    filtered_df['CVR - TY'] = np.where(((filtered_df['Bookings - TY'] != 0) & (filtered_df['Sessions - TY'] != 0)), (filtered_df['Bookings - TY']/filtered_df['Sessions - TY'] * 100), filtered_df['CVR - TY'])
    filtered_df['CVR - LY'] = np.where(((filtered_df['Bookings - LY'] != 0) & (filtered_df['Sessions - LY'] != 0)), (filtered_df['Bookings - LY']/filtered_df['Sessions - LY'] * 100), filtered_df['CVR - LY'])
    filtered_df['CVR YoY (Abs)'] =  np.where((filtered_df['CVR - TY'].notnull() & filtered_df['CVR - LY'].notnull()), ((filtered_df['CVR - TY'] - filtered_df['CVR - LY'])), filtered_df['CVR YoY (Abs)'])

    filtered_df['CPA - TY'] = np.where((filtered_df['Spend TY'] != 0) & (filtered_df['Bookings - TY'] != 0), (filtered_df['Spend TY']/filtered_df['Bookings - TY']), filtered_df['CPA - TY'])                     
    filtered_df['CPA - LY'] = np.where((filtered_df['Spend LY'] != 0) & (filtered_df['Bookings - LY'] != 0), (filtered_df['Spend LY']/filtered_df['Bookings - LY']), filtered_df['CPA - LY'])
    filtered_df['% YoY_CPA'] =  np.where((filtered_df['CPA - TY'] != 0) & (filtered_df['CPA - LY'] != 0), ((filtered_df['CPA - TY'] - filtered_df['CPA - LY'])/filtered_df['CPA - LY']) * 100, filtered_df['% YoY_CPA'])


    # Sessions Graphs 
    sessions_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Sessions - TY'],
      text='Sessions - TY'
    )
    sessions_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['Sessions - TY'],
      text='Sessions - LY'
    )
    sessions_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Sessions YoY (%)'],
      text='Sessions YoY (%)', opacity=0.6
    )
    # Spend Graphs
    spend_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Spend TY'],
      text='Spend TY'
    )
    spend_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['Spend TY'],
      text='Spend LY'
    )
    spend_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Spend YoY (%)'],
      text='Spend YoY (%)', opacity=0.6
    )
    # Bookings Graphs
    bookings_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Bookings - TY'],
      text='Bookings - TY'
    )
    bookings_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['Bookings - TY'],
      text='Bookings - LY'
    )
    bookings_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['Bookings - % - PY'],
      text='Bookings - % - PY', opacity=0.6
    )
    cpa_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['CPA - TY'],
      text='CPA - TY'
    )
    cpa_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['CPA - TY'],
      text='CPA - LY'
    )
    cpa_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['% YoY_CPA'],
      text='% CPA - YoY', opacity=0.6
    )
    cps_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['CPS - TY'],
      text='CPS - TY'
    )
    cps_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['CPS - TY'],
      text='CPS - LY'
    )
    cps_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['% YoY_CPS'],
      text='% CPS - YoY', opacity=0.6
    )
    cr_ty = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['CVR - TY'],
      text='CVR - TY'
    )
    cr_ly = go.Scatter(
      x=filtered_df[(filtered_df['Year'] == current_year-1)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year-1)]['CVR - TY'],
      text='CVR - LY'
    )
    cr_yoy = go.Bar(
      x=filtered_df[(filtered_df['Year'] == current_year)]['Week'],
      y=filtered_df[(filtered_df['Year'] == current_year)]['CVR YoY (Abs)'],
      text='CVR YoY (Abs)', opacity=0.6
    )

    fig = tools.make_subplots(
      rows=6, 
      cols=1, 
      shared_xaxes=True,
      subplot_titles=(								# Be sure to have same number of titles as number of graphs
        'Sessions', 
        'Spend',
        'Bookings', 
        'Cost per Acquisition',
        'CPS',
        'Conversion Rate'
        ))

    fig.append_trace(sessions_ty, 1, 1)		# 0
    fig.append_trace(sessions_ly, 1, 1)		# 1
    fig.append_trace(sessions_yoy, 1, 1)	# 2
    fig.append_trace(spend_ty, 2, 1)		# 3
    fig.append_trace(spend_ly, 2, 1)		# 4
    fig.append_trace(spend_yoy, 2, 1)		# 5
    fig.append_trace(bookings_ty, 3, 1)		# 6
    fig.append_trace(bookings_ly, 3, 1)		# 7
    fig.append_trace(bookings_yoy, 3, 1)	# 8
    fig.append_trace(cpa_ty, 4, 1)			# 9
    fig.append_trace(cpa_ly, 4, 1)			# 10
    fig.append_trace(cpa_yoy, 4, 1)			# 11
    fig.append_trace(cps_ty, 5, 1)			# 12
    fig.append_trace(cps_ly, 5, 1)			# 13
    fig.append_trace(cps_yoy, 5, 1)			# 14
    fig.append_trace(cr_ty, 6, 1)			# 15
    fig.append_trace(cr_ly, 6, 1)			# 16
    fig.append_trace(cr_yoy, 6, 1)			# 17

    # integer index below is the index of the trace
    # yaxis indices below need to start from the number of total graphs + 1 since they are on right-side
    # overlaing and anchor axes correspond to the graph number

    fig['data'][2].update(yaxis='y7')
    fig['layout']['yaxis7'] = dict(overlaying='y1', anchor='x1', side='right', showgrid=False, title='% Change YoY')

    fig['data'][5].update(yaxis='y8')
    fig['layout']['yaxis8'] = dict(overlaying='y2', anchor='x2', side='right', showgrid=False, title='% Change YoY')

    fig['data'][8].update(yaxis='y9')
    fig['layout']['yaxis9'] = dict(overlaying='y3', anchor='x3', side='right', showgrid=False, title='% Change YoY')

    fig['data'][11].update(yaxis='y10')
    fig['layout']['yaxis10'] = dict(overlaying='y4', anchor='x4', side='right', showgrid=False, title='% Change YoY')

    fig['data'][14].update(yaxis='y11')
    fig['layout']['yaxis11'] = dict(overlaying='y5', anchor='x5', side='right', showgrid=False, title='% Change YoY')

    fig['data'][17].update(yaxis='y12')
    fig['layout']['yaxis12'] = dict(overlaying='y6', anchor='x6', side='right', showgrid=False, title='% Change YoY')

    fig['layout']['xaxis'].update(title='Week of the Year' + ' - ' + str(current_year))
    for i in fig['layout']['annotations']:
      i['font'] = dict(size=12,
        # color='#ff0000'
        )
    fig['layout'].update(
      height= 1500,
      # width=750, 
      showlegend=False, 
      xaxis=dict(
        # tickmode='linear',
        # ticks='outside',
        # tick0=1,
        dtick=5,
        ticklen=8,
        tickwidth=2,
        tickcolor='#000',
        showgrid=True,
        zeroline=True,
        # showline=True,
        # mirror='ticks',
        # gridcolor='#bdbdbd',
        gridwidth=2
    ),
      )
    updated_fig = fig
    return updated_fig

columns_complete = ['Placement type', 'Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', \
                        'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)', \
                        'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', \
                        'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)']

columns_condensed = ['Placement type', 'Spend TY', 'Spend PoP (%)', 'Spend YoY (%)', 'Sessions - TY', 'Sessions PoP (%)', 'Sessions YoY (%)', \
                        'Bookings - TY',  'Bookings PoP (%)', 'Bookings YoY (%)',]

conditional_columns = ['Spend_PoP_abs_conditional', 'Spend_PoP_percent_conditional', 'Spend_YoY_percent_conditional',
'Sessions_PoP_percent_conditional', 'Sessions_YoY_percent_conditional', 
'Bookings_PoP_abs_conditional', 'Bookings_YoY_abs_conditional', 'Bookings_PoP_percent_conditional', 'Bookings_YoY_percent_conditional',
'Revenue_PoP_abs_conditional', 'Revenue_YoY_abs_conditional', 'Revenue_PoP_percent_conditional', 'Revenue_YoY_percent_conditional',]

dt_columns_total = ['Placement type', 'Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', \
                        'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)', \
                        'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', \
                        'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',
                        'Spend_PoP_abs_conditional', 'Spend_PoP_percent_conditional', 'Spend_YoY_percent_conditional',
'Sessions_PoP_percent_conditional', 'Sessions_YoY_percent_conditional', 
'Bookings_PoP_abs_conditional', 'Bookings_YoY_abs_conditional', 'Bookings_PoP_percent_conditional', 'Bookings_YoY_percent_conditional',
'Revenue_PoP_abs_conditional', 'Revenue_YoY_abs_conditional', 'Revenue_PoP_percent_conditional', 'Revenue_YoY_percent_conditional',]

######################## Birst Category Callbacks ######################## 

#### Date Picker Callback
@app.callback(Output('output-container-date-picker-range-birst-category', 'children'),
	[Input('my-date-picker-range-birst-category', 'start_date'),
	 Input('my-date-picker-range-birst-category', 'end_date')])
def update_output(start_date, end_date):
	string_prefix = 'You have selected '
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%B %d, %Y')
		string_prefix = string_prefix + 'a Start Date of ' + start_date_string + ' | '
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%B %d, %Y')
		days_selected = (end_date - start_date).days
		prior_start_date = start_date - timedelta(days_selected + 1)
		prior_start_date_string = datetime.strftime(prior_start_date, '%B %d, %Y')
		prior_end_date = end_date - timedelta(days_selected + 1)
		prior_end_date_string = datetime.strftime(prior_end_date, '%B %d, %Y')
		string_prefix = string_prefix + 'End Date of ' + end_date_string + ', for a total of ' + str(days_selected + 1) + ' Days. The prior period Start Date was ' + \
		prior_start_date_string + ' | End Date: ' + prior_end_date_string + '.'
	if len(string_prefix) == len('You have selected: '):
		return 'Select a date to see it displayed here'
	else:
		return string_prefix

# Callback and update first data table
@app.callback(Output('datatable-birst-category', 'data'),
	[Input('my-date-picker-range-birst-category', 'start_date'),
	 Input('my-date-picker-range-birst-category', 'end_date')])
def update_data_1(start_date, end_date):
	data_1 = update_first_datatable(start_date, end_date, None, 'Birst Category')
	return data_1

# Callback and update data table columns
@app.callback(Output('datatable-birst-category', 'columns'),
    [Input('radio-button-birst-category', 'value')])
def update_columns(value):
    if value == 'Complete':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_complete]
    elif value == 'Condensed':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_condensed]
    return column_set

# Callback for excel download
@app.callback(
    Output('download-link-birst-category', 'href'),
    [Input('my-date-picker-range-birst-category', 'start_date'),
	 Input('my-date-picker-range-birst-category', 'end_date')])   
def update_link(start_date, end_date):
	return '/cc-travel-report/birst-category/urlToDownload?value={}/{}'.format(dt.strptime(start_date,'%Y-%m-%d').strftime('%Y-%m-%d'),dt.strptime(end_date,'%Y-%m-%d').strftime('%Y-%m-%d'))
@app.server.route("/cc-travel-report/birst-category/urlToDownload") 
def download_excel_birst_category():
    value = flask.request.args.get('value')
    #here is where I split the value
    value = value.split('/')
    start_date = value[0]
    end_date = value[1]

    filename = datestamp + '_birst_category_' + start_date + '_to_' + end_date + '.xlsx'
	# Dummy Dataframe
    d = {'col1': [1, 2], 'col2': [3, 4]}
    df = pd.DataFrame(data=d)

    buf = io.BytesIO()
    excel_writer = pd.ExcelWriter(buf, engine="xlsxwriter")
    download_1 = update_first_download(start_date, end_date, None, 'Birst Category')
    download_1.to_excel(excel_writer, sheet_name="sheet1", index=False)
    # df.to_excel(excel_writer, sheet_name="sheet1", index=False)
    excel_writer.save()
    excel_data = buf.getvalue()
    buf.seek(0)

    return send_file(
        buf,
        mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 
        attachment_filename=filename,
        as_attachment=True,
        cache_timeout=0
    )

# Callback and update second data table
@app.callback(
	Output('datatable-birst-category-2', 'data'),
	[Input('my-date-picker-range-birst-category', 'start_date'),
	 Input('my-date-picker-range-birst-category', 'end_date')])
def update_data_2(start_date, end_date):
	data_2 = update_second_datatable(start_date, end_date, None, 'Birst Category')
	return data_2

# Callback for the Graphs
@app.callback(
   Output('birst-category', 'figure'),
   [Input('datatable-birst-category', "selected_rows"),
   Input('my-date-picker-range-birst-category', 'end_date')])
def update_birst_category(selected_rows, end_date):
	travel_product = []
	travel_product_list = sorted(df['Birst Category'].unique().tolist())
	for i in selected_rows:
		travel_product.append(travel_product_list[i])
		# Filter by specific product
	filtered_df = df[(df['Birst Category'].isin(travel_product))].groupby(['Year', 'Week']).sum()[['Spend TY', 'Spend LY', 'Sessions - TY', 'Sessions - LY', 'Bookings - TY', 'Bookings - LY', 'Revenue - TY', 'Revenue - LY']].reset_index()
	fig = update_graph(filtered_df, end_date)
	return fig

######################## GA Category Callbacks ######################## 

#### Date Picker Callback
@app.callback(Output('output-container-date-picker-range-ga-category', 'children'),
	[Input('my-date-picker-range-ga-category', 'start_date'),
	 Input('my-date-picker-range-ga-category', 'end_date')])
def update_output(start_date, end_date):
	string_prefix = 'You have selected '
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%B %d, %Y')
		string_prefix = string_prefix + 'a Start Date of ' + start_date_string + ' | '
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%B %d, %Y')
		days_selected = (end_date - start_date).days
		prior_start_date = start_date - timedelta(days_selected + 1)
		prior_start_date_string = datetime.strftime(prior_start_date, '%B %d, %Y')
		prior_end_date = end_date - timedelta(days_selected + 1)
		prior_end_date_string = datetime.strftime(prior_end_date, '%B %d, %Y')
		string_prefix = string_prefix + 'End Date of ' + end_date_string + ', for a total of ' + str(days_selected + 1) + ' Days. The prior period Start Date was ' + \
		prior_start_date_string + ' | End Date: ' + prior_end_date_string + '.'
	if len(string_prefix) == len('You have selected: '):
		return 'Select a date to see it displayed here'
	else:
		return string_prefix

# Callback and update first data table
@app.callback(Output('datatable-ga-category', 'data'),
	[Input('my-date-picker-range-ga-category', 'start_date'),
	 Input('my-date-picker-range-ga-category', 'end_date')])
def update_data_1(start_date, end_date):
	data_1 = update_first_datatable(start_date, end_date, None, 'GA Category')
	return data_1

# Callback and update data table columns
@app.callback(Output('datatable-ga-category', 'columns'),
    [Input('radio-button-ga-category', 'value')])
def update_columns(value):
    if value == 'Complete':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_complete]
    elif value == 'Condensed':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_condensed]
    return column_set

# Callback for excel download
@app.callback(
    Output('download-link-ga-category', 'href'),
    [Input('my-date-picker-range-ga-category', 'start_date'),
	 Input('my-date-picker-range-ga-category', 'end_date')])   
def update_link(start_date, end_date):
	return '/cc-travel-report/ga-category/urlToDownload?value={}/{}'.format(dt.strptime(start_date,'%Y-%m-%d').strftime('%Y-%m-%d'),dt.strptime(end_date,'%Y-%m-%d').strftime('%Y-%m-%d'))
@app.server.route("/cc-travel-report/ga-category/urlToDownload") 
def download_excel_ga_category():
    value = flask.request.args.get('value')
    #here is where I split the value
    value = value.split('/')
    start_date = value[0]
    end_date = value[1]

    filename = datestamp + '_ga_category_' + start_date + '_to_' + end_date + '.xlsx'
	# Dummy Dataframe
    d = {'col1': [1, 2], 'col2': [3, 4]}
    df = pd.DataFrame(data=d)

    buf = io.BytesIO()
    excel_writer = pd.ExcelWriter(buf, engine="xlsxwriter")
    download_1 = update_first_download(start_date, end_date, None, 'GA Category')
    download_1.to_excel(excel_writer, sheet_name="sheet1", index=False)
    # df.to_excel(excel_writer, sheet_name="sheet1", index=False)
    excel_writer.save()
    excel_data = buf.getvalue()
    buf.seek(0)

    return send_file(
        buf,
        mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 
        attachment_filename=filename,
        as_attachment=True,
        cache_timeout=0
    )

# Callback and update second data table
@app.callback(
	Output('datatable-ga-category-2', 'data'),
	[Input('my-date-picker-range-ga-category', 'start_date'),
	 Input('my-date-picker-range-ga-category', 'end_date')])
def update_data_2(start_date, end_date):
	data_2 = update_second_datatable(start_date, end_date, None, 'GA Category')
	return data_2

# Callback for the Graphs
@app.callback(
   Output('ga-category', 'figure'),
   [Input('datatable-ga-category', "selected_rows"),
   Input('my-date-picker-range-ga-category', 'end_date')])
def update_ga_category(selected_rows, end_date):
	travel_product = []
	travel_product_list = sorted(df['GA Category'].unique().tolist())
	for i in selected_rows:
		travel_product.append(travel_product_list[i])
		# Filter by specific product
	filtered_df = df[(df['GA Category'].isin(travel_product))].groupby(['Year', 'Week']).sum()[['Spend TY', 'Spend LY', 'Sessions - TY', 'Sessions - LY', 'Bookings - TY', 'Bookings - LY', 'Revenue - TY', 'Revenue - LY']].reset_index()
	fig = update_graph(filtered_df, end_date)
	return fig

######################## Paid Search Callbacks ######################## 

#### Date Picker Callback
@app.callback(Output('output-container-date-picker-range-paid-search', 'children'),
	[Input('my-date-picker-range-paid-search', 'start_date'),
	 Input('my-date-picker-range-paid-search', 'end_date')])
def update_output(start_date, end_date):
	string_prefix = 'You have selected '
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%B %d, %Y')
		string_prefix = string_prefix + 'a Start Date of ' + start_date_string + ' | '
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%B %d, %Y')
		days_selected = (end_date - start_date).days
		prior_start_date = start_date - timedelta(days_selected + 1)
		prior_start_date_string = datetime.strftime(prior_start_date, '%B %d, %Y')
		prior_end_date = end_date - timedelta(days_selected + 1)
		prior_end_date_string = datetime.strftime(prior_end_date, '%B %d, %Y')
		string_prefix = string_prefix + 'End Date of ' + end_date_string + ', for a total of ' + str(days_selected + 1) + ' Days. The prior period Start Date was ' + \
		prior_start_date_string + ' | End Date: ' + prior_end_date_string + '.'
	if len(string_prefix) == len('You have selected: '):
		return 'Select a date to see it displayed here'
	else:
		return string_prefix

# Callback and update first data table
@app.callback(Output('datatable-paid-search', 'data'),
	[Input('my-date-picker-range-paid-search', 'start_date'),
	 Input('my-date-picker-range-paid-search', 'end_date')])
def update_data_1(start_date, end_date):
	data_1 = update_first_datatable(start_date, end_date, 'Paid Search', 'Placement type')
	return data_1

# Callback and update data table columns
@app.callback(Output('datatable-paid-search', 'columns'),
    [Input('radio-button-paid-search', 'value')])
def update_columns(value):
    if value == 'Complete':
    	column_set=[{"name": i, "id": i, 'deletable': True} for i in columns_complete] + [{"name": j, "id": j, 'hidden': 'True'} for j in conditional_columns]
    elif value == 'Condensed':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_condensed]
    return column_set

# Callback for excel download
@app.callback(
    Output('download-link-paid-search-1', 'href'),
    [Input('my-date-picker-range-paid-search', 'start_date'),
	 Input('my-date-picker-range-paid-search', 'end_date')])   
def update_link(start_date, end_date):
	return '/cc-travel-report/paid-search/urlToDownload?value={}/{}'.format(dt.strptime(start_date,'%Y-%m-%d').strftime('%Y-%m-%d'),dt.strptime(end_date,'%Y-%m-%d').strftime('%Y-%m-%d'))
@app.server.route("/cc-travel-report/paid-search/urlToDownload") 
def download_excel_1():
    value = flask.request.args.get('value')
    #here is where I split the value
    value = value.split('/')
    start_date = value[0]
    end_date = value[1]

    filename = datestamp + '_paid_search_' + start_date + '_to_' + end_date + '.xlsx'
	# Dummy Dataframe
    d = {'col1': [1, 2], 'col2': [3, 4]}
    df = pd.DataFrame(data=d)

    buf = io.BytesIO()
    excel_writer = pd.ExcelWriter(buf, engine="xlsxwriter")
    download_1 = update_first_download(start_date, end_date, 'Paid Search', 'Placement type')
    download_1.to_excel(excel_writer, sheet_name="sheet1", index=False)
    # df.to_excel(excel_writer, sheet_name="sheet1", index=False)
    excel_writer.save()
    excel_data = buf.getvalue()
    buf.seek(0)

    return send_file(
        buf,
        mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 
        attachment_filename=filename,
        as_attachment=True,
        cache_timeout=0
    )

# Callback and update second data table
@app.callback(
	Output('datatable-paid-search-2', 'data'),
	[Input('my-date-picker-range-paid-search', 'start_date'),
	 Input('my-date-picker-range-paid-search', 'end_date')])
def update_data_2(start_date, end_date):
	data_2 = update_second_datatable(start_date, end_date, 'Paid Search', 'Placement type')
	return data_2

# Callback for the Graphs
@app.callback(
   Output('paid-search', 'figure'),
   [Input('datatable-paid-search', "selected_rows"),
   Input('my-date-picker-range-paid-search', 'end_date')])
def update_paid_search(selected_rows, end_date):
	travel_product = []
	travel_product_list = df[(df['Category'] == 'Paid Search')]['Placement type'].unique().tolist()
	for i in selected_rows:
		travel_product.append(travel_product_list[i])
		# Filter by specific product
	filtered_df = df[(df['Placement type'].isin(travel_product))].groupby(['Year', 'Week']).sum()[['Spend TY', 'Spend LY', 'Sessions - TY', 'Sessions - LY', 'Bookings - TY', 'Bookings - LY', 'Revenue - TY', 'Revenue - LY']].reset_index()
	fig = update_graph(filtered_df, end_date)
	return fig


######################## Display Callbacks ######################## 

#### Date Picker Callback
@app.callback(Output('output-container-date-picker-range-display', 'children'),
	[Input('my-date-picker-range-display', 'start_date'),
	 Input('my-date-picker-range-display', 'end_date')])
def update_output_display(start_date, end_date):
	string_prefix = 'You have selected '
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%B %d, %Y')
		string_prefix = string_prefix + 'a Start Date of ' + start_date_string + ' | '
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%B %d, %Y')
		days_selected = (end_date - start_date).days
		prior_start_date = start_date - timedelta(days_selected + 1)
		prior_start_date_string = datetime.strftime(prior_start_date, '%B %d, %Y')
		prior_end_date = end_date - timedelta(days_selected + 1)
		prior_end_date_string = datetime.strftime(prior_end_date, '%B %d, %Y')
		string_prefix = string_prefix + 'End Date of ' + end_date_string + ', for a total of ' + str(days_selected + 1) + ' Days. The prior period Start Date was ' + \
		prior_start_date_string + ' | End Date: ' + prior_end_date_string + '.'
	if len(string_prefix) == len('You have selected: '):
		return 'Select a date to see it displayed here'
	else:
		return string_prefix


# Callback and update first data table
@app.callback(Output('datatable-display', 'data'),
	[Input('my-date-picker-range-display', 'start_date'),
	 Input('my-date-picker-range-display', 'end_date')])
def update_data_1_display(start_date, end_date):
	data_1 = update_first_datatable(start_date, end_date, 'Display', 'Placement type')
	return data_1

# Callback and update data table columns
@app.callback(Output('datatable-display', 'columns'),
    [Input('radio-button-display', 'value')])
def update_columns(value):
    if value == 'Complete':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_complete]
    elif value == 'Condensed':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_condensed]
    return column_set

# Callback for excel download
@app.callback(
    Output('download-link-display-1', 'href'),
    [Input('my-date-picker-range-display', 'start_date'),
	 Input('my-date-picker-range-display', 'end_date')])   
def update_link(start_date, end_date):
	return '/cc-travel-report/display/urlToDownload?value={}/{}'.format(dt.strptime(start_date,'%Y-%m-%d').strftime('%Y-%m-%d'),dt.strptime(end_date,'%Y-%m-%d').strftime('%Y-%m-%d'))
@app.server.route("/cc-travel-report/display/urlToDownload") 
def download_excel_display_1():
    value = flask.request.args.get('value')
    #here is where I split the value
    value = value.split('/')
    start_date = value[0]
    end_date = value[1]

    filename = datestamp + '_display_' + start_date + '_to_' + end_date + '.xlsx'
	# Dummy Dataframe
    d = {'col1': [1, 2], 'col2': [3, 4]}
    df = pd.DataFrame(data=d)

    buf = io.BytesIO()
    excel_writer = pd.ExcelWriter(buf, engine="xlsxwriter")
    download_1 = update_first_download(start_date, end_date, 'Display', 'Placement type')
    download_1.to_excel(excel_writer, sheet_name="sheet1", index=False)
    # df.to_excel(excel_writer, sheet_name="sheet1", index=False)
    excel_writer.save()
    excel_data = buf.getvalue()
    buf.seek(0)

    return send_file(
        buf,
        mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 
        attachment_filename=filename,
        as_attachment=True,
        cache_timeout=0
    )

# Callback and update second data table
@app.callback(
	Output('datatable-display-2', 'data'),
	[Input('my-date-picker-range-display', 'start_date'),
	 Input('my-date-picker-range-display', 'end_date')])
def update_data_2_display(start_date, end_date):
	data_2 = update_second_datatable(start_date, end_date, 'Display', 'Placement type')
	return data_2

# Callback for the Graphs
@app.callback(
   Output('display', 'figure'),
   [Input('datatable-display', "selected_rows"),
   Input('my-date-picker-range-display', 'end_date')])
def update_display(selected_rows, end_date):
	travel_product = []
	travel_product_list = df[(df['Category'] == 'Display')]['Placement type'].unique().tolist()
	for i in selected_rows:
		travel_product.append(travel_product_list[i])
		# Filter by specific product
	filtered_df = df[(df['Placement type'].isin(travel_product))].groupby(['Year', 'Week']).sum()[['Spend TY', 'Spend LY', 'Sessions - TY', 'Sessions - LY', 'Bookings - TY', 'Bookings - LY', 'Revenue - TY', 'Revenue - LY']].reset_index()
	fig = update_graph(filtered_df, end_date)
	return fig

######################## Publishing Callbacks ######################## 

#### Date Picker Callback
@app.callback(Output('output-container-date-picker-range-publishing', 'children'),
	[Input('my-date-picker-range-publishing', 'start_date'),
	 Input('my-date-picker-range-publishing', 'end_date')])
def update_output_publishing(start_date, end_date):
	string_prefix = 'You have selected '
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%B %d, %Y')
		string_prefix = string_prefix + 'a Start Date of ' + start_date_string + ' | '
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%B %d, %Y')
		days_selected = (end_date - start_date).days
		prior_start_date = start_date - timedelta(days_selected + 1)
		prior_start_date_string = datetime.strftime(prior_start_date, '%B %d, %Y')
		prior_end_date = end_date - timedelta(days_selected + 1)
		prior_end_date_string = datetime.strftime(prior_end_date, '%B %d, %Y')
		string_prefix = string_prefix + 'End Date of ' + end_date_string + ', for a total of ' + str(days_selected + 1) + ' Days. The prior period Start Date was ' + \
		prior_start_date_string + ' | End Date: ' + prior_end_date_string + '.'
	if len(string_prefix) == len('You have selected: '):
		return 'Select a date to see it displayed here'
	else:
		return string_prefix

# Callback and update first data table
@app.callback(Output('datatable-publishing', 'data'),
	[Input('my-date-picker-range-publishing', 'start_date'),
	 Input('my-date-picker-range-publishing', 'end_date')])
def update_data_1_publishing(start_date, end_date):
	data_1 = update_first_datatable(start_date, end_date, 'Publishing', 'Placement type')
	return data_1

# Callback and update data table columns
@app.callback(Output('datatable-publishing', 'columns'),
    [Input('radio-button-publishing', 'value')])
def update_columns(value):
    if value == 'Complete':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_complete]
    elif value == 'Condensed':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_condensed]
    return column_set

# Callback for excel download
@app.callback(
    Output('download-link-publishing-1', 'href'),
    [Input('my-date-picker-range-publishing', 'start_date'),
	 Input('my-date-picker-range-publishing', 'end_date')])   
def update_link(start_date, end_date):
	return '/cc-travel-report/publishing/urlToDownload?value={}/{}'.format(dt.strptime(start_date,'%Y-%m-%d').strftime('%Y-%m-%d'),dt.strptime(end_date,'%Y-%m-%d').strftime('%Y-%m-%d'))
@app.server.route("/cc-travel-report/publishing/urlToDownload") 
def download_excel_publishing_1():
    value = flask.request.args.get('value')
    #here is where I split the value
    value = value.split('/')
    start_date = value[0]
    end_date = value[1]

    filename = datestamp + '_publishing_' + start_date + '_to_' + end_date + '.xlsx'
	# Dummy Dataframe
    d = {'col1': [1, 2], 'col2': [3, 4]}
    df = pd.DataFrame(data=d)

    buf = io.BytesIO()
    excel_writer = pd.ExcelWriter(buf, engine="xlsxwriter")
    download_1 = update_first_download(start_date, end_date, 'Publishing', 'Placement type')
    download_1.to_excel(excel_writer, sheet_name="sheet1", index=False)
    # df.to_excel(excel_writer, sheet_name="sheet1", index=False)
    excel_writer.save()
    excel_data = buf.getvalue()
    buf.seek(0)

    return send_file(
        buf,
        mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 
        attachment_filename=filename,
        as_attachment=True,
        cache_timeout=0
    )

# Callback and update second data table
@app.callback(
	Output('datatable-publishing-2', 'data'),
	[Input('my-date-picker-range-publishing', 'start_date'),
	 Input('my-date-picker-range-publishing', 'end_date')])
def update_data_2_publishing(start_date, end_date):
	data_2 = update_second_datatable(start_date, end_date, 'Publishing', 'Placement type')
	return data_2

# Callback for the Graphs
@app.callback(
   Output('publishing', 'figure'),
   [Input('datatable-publishing', "selected_rows"),
   Input('my-date-picker-range-publishing', 'end_date')])
def update_publishing(selected_rows, end_date):
	travel_product = []
	travel_product_list = df[(df['Category'] == 'Publishing')]['Placement type'].unique().tolist()
	for i in selected_rows:
		travel_product.append(travel_product_list[i])
		# Filter by specific product
	filtered_df = df[(df['Placement type'].isin(travel_product))].groupby(['Year', 'Week']).sum()[['Spend TY', 'Spend LY', 'Sessions - TY', 'Sessions - LY', 'Bookings - TY', 'Bookings - LY', 'Revenue - TY', 'Revenue - LY']].reset_index()
	fig = update_graph(filtered_df, end_date)
	return fig

######################## Metasearch Callbacks ######################## 

#### Date Picker Callback
@app.callback(Output('output-container-date-picker-range-metasearch', 'children'),
	[Input('my-date-picker-range-metasearch', 'start_date'),
	 Input('my-date-picker-range-metasearch', 'end_date')])
def update_output_metasearch(start_date, end_date):
	string_prefix = 'You have selected '
	if start_date is not None:
		start_date = dt.strptime(start_date, '%Y-%m-%d')
		start_date_string = start_date.strftime('%B %d, %Y')
		string_prefix = string_prefix + 'a Start Date of ' + start_date_string + ' | '
	if end_date is not None:
		end_date = dt.strptime(end_date, '%Y-%m-%d')
		end_date_string = end_date.strftime('%B %d, %Y')
		days_selected = (end_date - start_date).days
		prior_start_date = start_date - timedelta(days_selected + 1)
		prior_start_date_string = datetime.strftime(prior_start_date, '%B %d, %Y')
		prior_end_date = end_date - timedelta(days_selected + 1)
		prior_end_date_string = datetime.strftime(prior_end_date, '%B %d, %Y')
		string_prefix = string_prefix + 'End Date of ' + end_date_string + ', for a total of ' + str(days_selected + 1) + ' Days. The prior period Start Date was ' + \
		prior_start_date_string + ' | End Date: ' + prior_end_date_string + '.'
	if len(string_prefix) == len('You have selected: '):
		return 'Select a date to see it displayed here'
	else:
		return string_prefix

# Callback and update first data table
@app.callback(Output('datatable-metasearch', 'data'),
	[Input('my-date-picker-range-metasearch', 'start_date'),
	 Input('my-date-picker-range-metasearch', 'end_date')])
def update_data_1_metasearch(start_date, end_date):
	data_1 = update_first_datatable(start_date, end_date, 'Metasearch and Travel Ads', 'Placement type')
	return data_1

# Callback and update data table columns
@app.callback(Output('datatable-metasearch', 'columns'),
    [Input('radio-button-metasearch', 'value')])
def update_columns(value):
    if value == 'Complete':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_complete]
    elif value == 'Condensed':
        column_set=[{"name": i, "id": i, "deletable": True} for i in columns_condensed]
    return column_set

# Callback for excel download
@app.callback(
    Output('download-link-metasearch-1', 'href'),
    [Input('my-date-picker-range-metasearch', 'start_date'),
	 Input('my-date-picker-range-metasearch', 'end_date')])   
def update_link(start_date, end_date):
	return '/cc-travel-report/metasearch/urlToDownload?value={}/{}'.format(dt.strptime(start_date,'%Y-%m-%d').strftime('%Y-%m-%d'),dt.strptime(end_date,'%Y-%m-%d').strftime('%Y-%m-%d'))
@app.server.route("/cc-travel-report/metasearch/urlToDownload") 
def download_excel_metasearch_1():
    value = flask.request.args.get('value')
    #here is where I split the value
    value = value.split('/')
    start_date = value[0]
    end_date = value[1]

    filename = datestamp + '_metasearch_' + start_date + '_to_' + end_date + '.xlsx'
	# Dummy Dataframe
    d = {'col1': [1, 2], 'col2': [3, 4]}
    df = pd.DataFrame(data=d)

    buf = io.BytesIO()
    excel_writer = pd.ExcelWriter(buf, engine="xlsxwriter")
    download_1 = update_first_download(start_date, end_date, 'Metasearch and Travel Ads', 'Placement type')
    download_1.to_excel(excel_writer, sheet_name="sheet1", index=False)
    # df.to_excel(excel_writer, sheet_name="sheet1", index=False)
    excel_writer.save()
    excel_data = buf.getvalue()
    buf.seek(0)

    return send_file(
        buf,
        mimetype = 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet', 
        attachment_filename=filename,
        as_attachment=True,
        cache_timeout=0
    )

# Callback and update second data table
@app.callback(
	Output('datatable-metasearch-2', 'data'),
	[Input('my-date-picker-range-metasearch', 'start_date'),
	 Input('my-date-picker-range-metasearch', 'end_date')])
def update_data_2_metasearch(start_date, end_date):
	data_2 = update_second_datatable(start_date, end_date, 'Metasearch and Travel Ads', 'Placement type')
	return data_2

# Callback for the Graphs
@app.callback(
   Output('metasearch', 'figure'),
   [Input('datatable-metasearch', "selected_rows"),
	 Input('my-date-picker-range-metasearch', 'end_date')])
def update_metasearch(selected_rows, end_date):
	travel_product = []
	travel_product_list = df[(df['Category'] == 'Metasearch and Travel Ads')]['Placement type'].unique().tolist()
	for i in selected_rows:
		travel_product.append(travel_product_list[i])
		# Filter by specific product
	filtered_df = df[(df['Placement type'].isin(travel_product))].groupby(['Year', 'Week']).sum()[['Spend TY', 'Spend LY', 'Sessions - TY', 'Sessions - LY', 'Bookings - TY', 'Bookings - LY', 'Revenue - TY', 'Revenue - LY']].reset_index()
	fig = update_graph(filtered_df, end_date)
	return fig

ModuleNotFoundError: No module named 'components'

In [4]:
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
# see https://community.plot.ly/t/nolayoutexception-on-deployment-of-multi-page-dash-app-example-code/12463/2?u=dcomfort
from app import server
from app import app
from layouts import layout_birst_category, layout_ga_category, layout_paid_search, noPage, layout_display, layout_publishing, layout_metasearch
import callbacks

import pandas as pd
import io
import xlsxwriter
from flask import send_file

# see https://dash.plot.ly/external-resources to alter header, footer and favicon
app.index_string = ''' 
<!DOCTYPE html>
<html>
    <head>
        {%metas%}
        <title>Stark Performance Marketing Report</title>
        {%favicon%}
        {%css%}
    </head>
    <body>
        {%app_entry%}
        <footer>
            {%config%}
            {%scripts%}
        </footer>
        <div>Stark Performance Marketing Report</div>
    </body>
</html>
'''

app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Update page
# # # # # # # # #
@app.callback(Output('page-content', 'children'),
              [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/cc-travel-report' or pathname == '/cc-travel-report/overview-birst/':
        return layout_birst_category
    elif pathname == '/cc-travel-report/overview-ga/':
        return layout_ga_category
    elif pathname == '/cc-travel-report/paid-search/':
        return layout_paid_search
    elif pathname == '/cc-travel-report/display/':
        return layout_display
    elif pathname == '/cc-travel-report/publishing/':
        return layout_publishing
    elif pathname == '/cc-travel-report/metasearch-and-travel-ads/':
        return layout_metasearch
    else:
        return noPage

# # # # # # # # #
# detail the way that external_css and external_js work and link to alternative method locally hosted
# # # # # # # # #
external_css = ["https://cdnjs.cloudflare.com/ajax/libs/normalize/7.0.0/normalize.min.css",
                "https://cdnjs.cloudflare.com/ajax/libs/skeleton/2.0.4/skeleton.min.css",
                "//fonts.googleapis.com/css?family=Raleway:400,300,600",
                "https://codepen.io/bcd/pen/KQrXdb.css",
                "https://maxcdn.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css",
                "https://codepen.io/dmcomfort/pen/JzdzEZ.css"]

for css in external_css:
    app.css.append_css({"external_url": css})

external_js = ["https://code.jquery.com/jquery-3.2.1.min.js",
               "https://codepen.io/bcd/pen/YaXojL.js"]

for js in external_js:
    app.scripts.append_script({"external_url": js})

if __name__ == '__main__':
    app.run_server(debug=False)

ModuleNotFoundError: No module named 'app'

In [5]:
import dash_core_components as dcc
import dash_html_components as html
import dash_table
from components import Header, print_button
from datetime import datetime as dt
from datetime import date, timedelta
import pandas as pd


# Read in Travel Report Data
df = pd.read_csv('data/performance_analytics_cost_and_ga_metrics.csv')

df.rename(columns={
 'Travel Product': 'Placement type', 
  'Spend - This Year': 'Spend TY', 
  'Spend - Last Year': 'Spend LY', 
  'Sessions - This Year': 'Sessions - TY',
  'Sessions - Last Year': 'Sessions - LY',
  'Bookings - This Year': 'Bookings - TY',
  'Bookings - Last Year': 'Bookings - LY',
  'Revenue - This Year': 'Revenue - TY',
  'Revenue - Last Year': 'Revenue - LY',
  }, inplace=True)


df['Date'] = pd.to_datetime(df['Date'])
current_year = df['Year'].max()

dt_columns = ['Placement type', 'Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', \
                        'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)', \
                        'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', \
                        'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',]

conditional_columns = ['Spend_PoP_abs_conditional', 'Spend_PoP_percent_conditional', 'Spend_YoY_percent_conditional',
'Sessions_PoP_percent_conditional', 'Sessions_YoY_percent_conditional', 
'Bookings_PoP_abs_conditional', 'Bookings_YoY_abs_conditional', 'Bookings_PoP_percent_conditional', 'Bookings_YoY_percent_conditional',
'Revenue_PoP_abs_conditional', 'Revenue_YoY_abs_conditional', 'Revenue_PoP_percent_conditional', 'Revenue_YoY_percent_conditional',]

dt_columns_total = ['Placement type', 'Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', \
                        'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)', \
                        'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', \
                        'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',
                        'Spend_PoP_abs_conditional', 'Spend_PoP_percent_conditional', 'Spend_YoY_percent_conditional',
'Sessions_PoP_percent_conditional', 'Sessions_YoY_percent_conditional', 
'Bookings_PoP_abs_conditional', 'Bookings_YoY_abs_conditional', 'Bookings_PoP_percent_conditional', 'Bookings_YoY_percent_conditional',
'Revenue_PoP_abs_conditional', 'Revenue_YoY_abs_conditional', 'Revenue_PoP_percent_conditional', 'Revenue_YoY_percent_conditional',]

df_columns_calculated = ['Placement type', 'CPS - TY', 
                        'CPS - LP', 'CPS PoP (Abs)', 'CPS PoP (%)',
                        'CPS - LY',  'CPS YoY (Abs)',  'CPS YoY (%)', 
                        'CVR - TY', 
                        'CVR - LP', 'CVR PoP (Abs)', 'CVR PoP (%)',
                        'CVR - LY',  'CVR YoY (Abs)',  'CVR YoY (%)',
                        'CPA - TY', 
                        'CPA - LP', 'CPA PoP (Abs)', 'CPA PoP (%)',
                        'CPA - LY', 'CPA YoY (Abs)',  'CPA YoY (%)']

conditional_columns_calculated_calculated = ['CPS_PoP_abs_conditional', 'CPS_PoP_percent_conditional', 'CPS_YoY_abs_conditional', 'CPS_PoP_percent_conditional', 
'CVR_PoP_abs_conditional', 'CVR_PoP_percent_conditional', 'CVR_YoY_abs_conditional', 'CVR_YoY_percent_conditional',
'CPA_PoP_abs_conditional', 'CPA_PoP_percent_conditional', 'CPA_YoY_abs_conditional', 'CPA_YoY_percent_conditional']

######################## START Birst Category Layout ########################
layout_birst_category =  html.Div([

#    print_button(),
 
    html.Div([
        # CC Header
        Header(),
        # Date Picker
        html.Div([
            dcc.DatePickerRange(
              id='my-date-picker-range-birst-category',
              # with_portal=True,
              min_date_allowed=dt(2018, 1, 1),
              max_date_allowed=df['Date'].max().to_pydatetime(),
              initial_visible_month=dt(current_year,df['Date'].max().to_pydatetime().month, 1),
              start_date=(df['Date'].max() - timedelta(6)).to_pydatetime(),
              end_date=df['Date'].max().to_pydatetime(),
            ),
            html.Div(id='output-container-date-picker-range-birst-category')
            ], className="row ", style={'marginTop': 30, 'marginBottom': 15}),
        # Header Bar
        html.Div([
          html.H6(["Birst Level Metrics"], className="gs-header gs-text-header padded",style={'marginTop': 15})
          ]),
        # Radio Button
        html.Div([
          dcc.RadioItems(
            options=[
                {'label': 'Condensed Data Table', 'value': 'Condensed'},
                {'label': 'Complete Data Table', 'value': 'Complete'},
            ], value='Condensed',
            labelStyle={'display': 'inline-block', 'width': '20%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 15},
            id='radio-button-birst-category'
            )]),
        # First Data Table
        html.Div([
            dash_table.DataTable(
                id='datatable-birst-category',
                columns=[{"name": i, "id": i, 'deletable': True} for i in dt_columns],
                editable=True,
                n_fixed_columns=2,
                style_table={'maxWidth': '1500px'},
                row_selectable="multi",
                selected_rows=[0],
                style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                css=[{'selector': '.dash-cell div.dash-cell-value', 'rule': 'display: inline; white-space: inherit; overflow: inherit; text-overflow: inherit;'}],
                style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]  
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EAFAF1'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D5F5E3'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FEF9E7'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FCF3CF'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EBF5FB'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D6EAF8'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c},'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF' } for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)','CPA PoP (%)', 'CPA YoY (%)' ]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c},'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866' } for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', 'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 
                      'Sessions YoY (%)', 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',]] 
                ),
            ], className=" twelve columns"),
        # Download Button
        html.Div([
          html.A(html.Button('Download Data', id='download-button'), id='download-link-birst-category')
          ]),
        # Second Data Table
        html.Div([
            dash_table.DataTable(
              id='datatable-birst-category-2',
              columns=[{"name": i, "id": i} for i in df_columns_calculated],
              editable=True,
              n_fixed_columns=1,
              style_table={'maxWidth': '1500px'},
                  # sorting=True,
                  # sorting_type="multi",
                   style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                   style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', 'CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)', 'CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]],
            ),
            ], className=" twelve columns"),
        # GRAPHS
        html.Div([
            html.Div(
            id='update_graph_1'
            ),
            html.Div([
                dcc.Graph(id='birst-category'),
            ], className=" twelve columns"
            ),], className="row "
        ),
        ], className="subpage")
    ], className="page")

######################## END Birst Category Layout ########################

######################## START GA Category Layout ########################
layout_ga_category =  html.Div([
    html.Div([
        # CC Header
        Header(),
        # Date Picker
        html.Div([
            dcc.DatePickerRange(
              id='my-date-picker-range-ga-category',
              # with_portal=True,
              min_date_allowed=dt(2018, 1, 1),
              max_date_allowed=df['Date'].max().to_pydatetime(),
              initial_visible_month=dt(current_year,df['Date'].max().to_pydatetime().month, 1),
              start_date=(df['Date'].max() - timedelta(6)).to_pydatetime(),
              end_date=df['Date'].max().to_pydatetime(),
            ),
            html.Div(id='output-container-date-picker-range-ga-category')
            ], className="row ", style={'marginTop': 30, 'marginBottom': 15}),
        # Header Bar
        html.Div([
          html.H6(["GA Level Metrics"], className="gs-header gs-text-header padded",style={'marginTop': 15})
          ]),
        # Radio Button
        html.Div([
          dcc.RadioItems(
            options=[
                {'label': 'Condensed Data Table', 'value': 'Condensed'},
                {'label': 'Complete Data Table', 'value': 'Complete'},
            ], value='Condensed',
            labelStyle={'display': 'inline-block', 'width': '20%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 15},
            id='radio-button-ga-category'
            )]),
        # First Data Table
        html.Div([
            dash_table.DataTable(
                id='datatable-ga-category',
                columns=[{"name": i, "id": i, 'deletable': True} for i in dt_columns],
                editable=True,
                n_fixed_columns=2,
                style_table={'maxWidth': '1500px'},
                row_selectable="multi",
                selected_rows=[0],
                style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                css=[{'selector': '.dash-cell div.dash-cell-value', 'rule': 'display: inline; white-space: inherit; overflow: inherit; text-overflow: inherit;'}],
                style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]  
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EAFAF1'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D5F5E3'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FEF9E7'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FCF3CF'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EBF5FB'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D6EAF8'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c},'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF' } for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)','CPA PoP (%)', 'CPA YoY (%)' ]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c},'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866' } for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', 'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 
                      'Sessions YoY (%)', 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',]],
                ),
            ], className=" twelve columns"),
        # Download Button
        html.Div([
          html.A(html.Button('Download Data', id='download-button'), id='download-link-ga-category')
          ]),
        # Second Data Table
        html.Div([
            dash_table.DataTable(
              id='datatable-ga-category-2',
              columns=[{"name": i, "id": i} for i in df_columns_calculated],
              editable=True,
              n_fixed_columns=1,
              style_table={'maxWidth': '1500px'},
                  # sorting=True,
                  # sorting_type="multi",
                   style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                   style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', 'CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)', 'CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]],
            ),
            ], className=" twelve columns"),
        # GRAPHS
        html.Div([
          html.Div(
            id='update_graph_1'
            ),
            html.Div([
                dcc.Graph(id='ga-category'),
            ], className=" twelve columns"
            ),], className="row "
        ),
        ], className="subpage")
    ], className="page")

######################## END GA Category Layout ########################

######################## START Paid Search Layout ########################
layout_paid_search =  html.Div([
    html.Div([
        # CC Header
        Header(),
        # Date Picker
        html.Div([
            dcc.DatePickerRange(
              id='my-date-picker-range-paid-search',
              min_date_allowed=dt(2018, 1, 1),
              max_date_allowed=df['Date'].max().to_pydatetime(),
              initial_visible_month=dt(current_year,df['Date'].max().to_pydatetime().month, 1),
              start_date=(df['Date'].max() - timedelta(6)).to_pydatetime(),
              end_date=df['Date'].max().to_pydatetime(),
            ),
            html.Div(id='output-container-date-picker-range-paid-search')
            ], className="row ", style={'marginTop': 30, 'marginBottom': 15}),
        # Header Bar
        html.Div([
          html.H6(["Paid Search"], className="gs-header gs-text-header padded",style={'marginTop': 15})
          ]),
        # Radio Button
        html.Div([
          dcc.RadioItems(
            options=[
                {'label': 'Condensed Data Table', 'value': 'Condensed'},
                {'label': 'Complete Data Table', 'value': 'Complete'},
            ], value='Condensed',
            labelStyle={'display': 'inline-block', 'width': '20%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 15},
            id='radio-button-paid-search'
            )]),
        # First Data Table
        html.Div([
            dash_table.DataTable(
                id='datatable-paid-search',
                columns=[{"name": i, "id": i, 'deletable': True} for i in dt_columns] 
                + [{"name": j, "id": j, 'hidden': 'True'} for j in conditional_columns], 
                editable=True,
                n_fixed_columns=2,
                style_table={'maxWidth': '1500px'},
                row_selectable="multi",
                selected_rows=[0],
                style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                css=[{'selector': '.dash-cell div.dash-cell-value', 'rule': 'display: inline; white-space: inherit; overflow: inherit; text-overflow: inherit;'}],
                style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]  
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EAFAF1'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D5F5E3'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FEF9E7'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FCF3CF'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EBF5FB'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D6EAF8'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c},'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF' } for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)','CPA PoP (%)', 'CPA YoY (%)' ]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c},'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866' } for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', 'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 
                      'Sessions YoY (%)', 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',]]
                      + [{'if': {'column_id': 'Spend PoP (Abs)', 'filter': 'Spend_PoP_abs_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Spend PoP (%)', 'filter': 'Spend_PoP_percent_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Spend YoY (%)', 'filter': 'Spend_YoY_percent_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Sessions PoP (%)', 'filter': 'Sessions_PoP_percent_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Sessions YoY (%)', 'filter': 'Sessions_YoY_percent_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Bookings PoP (Abs)', 'filter': 'Bookings_PoP_abs_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Bookings YoY (Abs)', 'filter': 'Bookings_YoY_abs_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Bookings PoP (%)', 'filter': 'Bookings_PoP_percent_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Bookings YoY (%)', 'filter': 'Bookings_YoY_percent_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Revenue PoP (Abs)', 'filter': 'Revenue_PoP_abs_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Revenue YoY (Abs)', 'filter': 'Revenue_YoY_abs_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Revenue PoP (%)', 'filter': 'Revenue_PoP_percent_conditional < num(0)'}, 'color': 'red'}]
                      + [{'if': {'column_id': 'Revenue YoY (%)', 'filter': 'Revenue_YoY_percent_conditional < num(0)'}, 'color': 'red'}],
                      style_header={'backgroundColor': 'black','color': 'white'},
                ),
            ], className=" twelve columns"),
        # Download Button
        html.Div([
          html.A(html.Button('Download Data', id='download-button'), id='download-link-paid-search-1')
          ]),
        # Second Data Table
        html.Div([
            dash_table.DataTable(
              id='datatable-paid-search-2',
              columns=[{"name": i, "id": i} for i in df_columns_calculated] + 
              [{"name": k, "id": k, 'hidden': 'True'} for k in conditional_columns_calculated_calculated],
              editable=True,
              n_fixed_columns=1,
              css=[{'selector': '.dash-cell div.dash-cell-value', 'rule': 'display: inline; white-space: inherit; overflow: inherit; text-overflow: inherit;'}],
              style_table={'maxWidth': '1500px'},
              style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
              style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]
                + [{'if': {'column_id': c},  'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                + [{'if': {'column_id': c},  'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', 'CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)', 'CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                + [{'if': {'column_id': 'CPS PoP (Abs)', 'filter': 'CPS_PoP_abs_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CPS_PoP_abs_conditional', 'filter': 'CPS_PoP_abs_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CPS PoP (%)', 'filter': 'CPS_PoP_percent_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CPS YoY (Abs)', 'filter': 'CPS_YoY_abs_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CPS YoY (%)', 'filter': 'CPS_PoP_percent_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CVR PoP (Abs)', 'filter': 'CVR_PoP_abs_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CVR PoP (%)', 'filter': 'CVR_PoP_percent_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id':  'CVR YoY (Abs)', 'filter': 'CVR_YoY_abs_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CVR YoY (%)', 'filter': 'CVR_YoY_percent_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CPA PoP (Abs)', 'filter': 'CPA_PoP_abs_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CPA PoP (%)', 'filter': 'CPA_PoP_percent_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CPA YoY (Abs)', 'filter': 'CPA_YoY_abs_conditional < num(0)'}, 'color': 'red'}]
                + [{'if': {'column_id': 'CPA YoY (%)', 'filter': 'CPA_YoY_percent_conditional < num(0)'}, 'color': 'red'}],
                style_header={'backgroundColor': 'black','color': 'white'},       
                ),
            ], className=" twelve columns"),
        # GRAPHS
        html.Div([
            html.Div([
              dcc.Graph(id='paid-search'),
              ], className=" twelve columns"
              )
            ], className="row ")
        ], className="subpage")
    ], className="page")

######################## END Paid Search Layout ########################

######################## START Display Layout ########################
layout_display =  html.Div([
    html.Div([
        # CC Header
        Header(),
        # Date Picker
        html.Div([
            dcc.DatePickerRange(
              id='my-date-picker-range-display',
              # with_portal=True,
              min_date_allowed=dt(2018, 1, 1),
              max_date_allowed=df['Date'].max().to_pydatetime(),
              initial_visible_month=dt(current_year,df['Date'].max().to_pydatetime().month, 1),
              start_date=(df['Date'].max() - timedelta(6)).to_pydatetime(),
              end_date=df['Date'].max().to_pydatetime(),
            ),
            html.Div(id='output-container-date-picker-range-display')
            ], className="row ", style={'marginTop': 30, 'marginBottom': 15}),
        # Header Bar
        html.Div([
          html.H6(["Display"], className="gs-header gs-text-header padded",style={'marginTop': 15})
          ]),
        # Radio Button
        html.Div([
          dcc.RadioItems(
            options=[
                {'label': 'Condensed Data Table', 'value': 'Condensed'},
                {'label': 'Complete Data Table', 'value': 'Complete'},
            ], value='Condensed',
            labelStyle={'display': 'inline-block', 'width': '20%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 15},
            id='radio-button-display'
            )]),
        # First Data Table
        html.Div([
            dash_table.DataTable(
                id='datatable-display',
                columns=[{"name": i, "id": i, 'deletable': True} for i in dt_columns],
                editable=True,
                n_fixed_columns=2,
                style_table={'maxWidth': '1500px'},
                row_selectable="multi",
                selected_rows=[0],
                style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                css=[{'selector': '.dash-cell div.dash-cell-value', 'rule': 'display: inline; white-space: inherit; overflow: inherit; text-overflow: inherit;'}],
                style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]  
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EAFAF1'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D5F5E3'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FEF9E7'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FCF3CF'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EBF5FB'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D6EAF8'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c},'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF' } for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)','CPA PoP (%)', 'CPA YoY (%)' ]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c},'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866' } for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', 'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 
                      'Sessions YoY (%)', 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',]],
                ),
            ], className=" twelve columns"),
        # Download Button
        html.Div([
          html.A(html.Button('Download Data', id='download-button'), id='download-link-display-1')
          ]),
        # Second Data Table
        html.Div([
            dash_table.DataTable(
              id='datatable-display-2',
              columns=[{"name": i, "id": i} for i in df_columns_calculated],
              editable=True,
              n_fixed_columns=1,
              style_table={'maxWidth': '1500px'},
                  # sorting=True,
                  # sorting_type="multi",
                   style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                   style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', 'CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)', 'CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]],
            ),
            ], className=" twelve columns"),
        # GRAPHS
        html.Div([
            html.Div(
            id='update_graph_1'
            ),
            html.Div([
                dcc.Graph(id='display'),
            ], className=" twelve columns"
            ),], className="row "
        ),
        ], className="subpage")
    ], className="page")

######################## END Display Layout ########################

######################## START Publishing Layout ########################
layout_publishing =  html.Div([
    html.Div([
        # CC Header
        Header(),
        # Date Picker
        html.Div([
            dcc.DatePickerRange(
              id='my-date-picker-range-publishing',
              # with_portal=True,
              min_date_allowed=dt(2018, 1, 1),
              max_date_allowed=df['Date'].max().to_pydatetime(),
              initial_visible_month=dt(current_year,df['Date'].max().to_pydatetime().month, 1),
              start_date=(df['Date'].max() - timedelta(6)).to_pydatetime(),
              end_date=df['Date'].max().to_pydatetime(),
            ),
            html.Div(id='output-container-date-picker-range-publishing')
            ], className="row ", style={'marginTop': 30, 'marginBottom': 15}),
        # Header Bar
        html.Div([
          html.H6(["Publishing"], className="gs-header gs-text-header padded",style={'marginTop': 15})
          ]),
        # Radio Button
        html.Div([
          dcc.RadioItems(
            options=[
                {'label': 'Condensed Data Table', 'value': 'Condensed'},
                {'label': 'Complete Data Table', 'value': 'Complete'},
            ], value='Condensed',
            labelStyle={'display': 'inline-block', 'width': '20%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 15},
            id='radio-button-publishing'
            )]),
        # First Data Table
        html.Div([
            dash_table.DataTable(
                id='datatable-publishing',
                columns=[{"name": i, "id": i, 'deletable': True} for i in dt_columns],
                editable=True,
                n_fixed_columns=2,
                style_table={'maxWidth': '1500px'},
                row_selectable="multi",
                selected_rows=[0],
                style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                css=[{'selector': '.dash-cell div.dash-cell-value', 'rule': 'display: inline; white-space: inherit; overflow: inherit; text-overflow: inherit;'}],
                style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]  
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EAFAF1'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D5F5E3'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FEF9E7'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FCF3CF'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EBF5FB'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D6EAF8'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c},'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF' } for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)','CPA PoP (%)', 'CPA YoY (%)' ]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c},'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866' } for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', 'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 
                      'Sessions YoY (%)', 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',]],
                ),
            ], className=" twelve columns"),
        # Download Button
        html.Div([
          html.A(html.Button('Download Data', id='download-button'), id='download-link-publishing-1')
          ]),
        # Second Data Table
        html.Div([
            dash_table.DataTable(
              id='datatable-publishing-2',
              columns=[{"name": i, "id": i} for i in df_columns_calculated],
              editable=True,
              n_fixed_columns=1,
              style_table={'maxWidth': '1500px'},
                  # sorting=True,
                  # sorting_type="multi",
                   style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                   style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', 'CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)', 'CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]],
            ),
            ], className=" twelve columns"),
        # GRAPHS
        html.Div([
            html.Div(
            id='update_graph_1'
            ),
            html.Div([
                dcc.Graph(id='publishing'),
            ], className=" twelve columns"
            ),], className="row "
        ),
        ], className="subpage")
    ], className="page")

######################## END Publishing Layout ########################

######################## START Metasearch and Travel Ads Layout ########################
layout_metasearch =  html.Div([
    html.Div([
        # CC Header
        Header(),
        # Date Picker
        html.Div([
            dcc.DatePickerRange(
              id='my-date-picker-range-metasearch',
              # with_portal=True,
              min_date_allowed=dt(2018, 1, 1),
              max_date_allowed=df['Date'].max().to_pydatetime(),
              initial_visible_month=dt(current_year,df['Date'].max().to_pydatetime().month, 1),
              start_date=(df['Date'].max() - timedelta(6)).to_pydatetime(),
              end_date=df['Date'].max().to_pydatetime(),
            ),
            html.Div(id='output-container-date-picker-range-metasearch')
            ], className="row ", style={'marginTop': 30, 'marginBottom': 15}),
        # Header Bar
        html.Div([
          html.H6(["Metasearch and Travel Ads"], className="gs-header gs-text-header padded",style={'marginTop': 15})
          ]),
        # Radio Button
        html.Div([
          dcc.RadioItems(
            options=[
                {'label': 'Condensed Data Table', 'value': 'Condensed'},
                {'label': 'Complete Data Table', 'value': 'Complete'},
            ], value='Condensed',
            labelStyle={'display': 'inline-block', 'width': '20%', 'margin':'auto', 'marginTop': 15, 'paddingLeft': 15},
            id='radio-button-metasearch'
            )]),
        # First Data Table
        html.Div([
            dash_table.DataTable(
                id='datatable-metasearch',
                columns=[{"name": i, "id": i, 'deletable': True} for i in dt_columns],
                editable=True,
                n_fixed_columns=2,
                style_table={'maxWidth': '1500px'},
                row_selectable="multi",
                selected_rows=[0],
                style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                css=[{'selector': '.dash-cell div.dash-cell-value', 'rule': 'display: inline; white-space: inherit; overflow: inherit; text-overflow: inherit;'}],
                style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]  
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EAFAF1'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D5F5E3'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)',]] 
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FEF9E7'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]] 
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FCF3CF'} for c in ['Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 'Sessions YoY (%)',]]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#EBF5FB'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#D6EAF8'} for c in ['Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)',]]
                      + [{'if': {'column_id': c},'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF' } for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY',  'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                      + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)','CPA PoP (%)', 'CPA YoY (%)' ]]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8' } for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c},'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866' } for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY',  'CPS YoY (Abs)', 'CPS PoP (%)', 'CPA YoY (%)']]
                      + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['Spend TY', 'Spend - LP', 'Spend PoP (Abs)', 'Spend PoP (%)', 'Spend LY', 'Spend YoY (%)', 'Sessions - TY', 'Sessions - LP', 'Sessions - LY', 'Sessions PoP (%)', 
                      'Sessions YoY (%)', 'Bookings - TY', 'Bookings - LP', 'Bookings PoP (%)', 'Bookings PoP (Abs)', 'Bookings - LY', 'Bookings YoY (%)', 'Bookings YoY (Abs)', 'Revenue - TY', 'Revenue - LP', 'Revenue PoP (Abs)', 'Revenue PoP (%)', 'Revenue - LY', 'Revenue YoY (%)', 'Revenue YoY (Abs)',]],
                ),
            ], className=" twelve columns"),
        # Download Button
        html.Div([
          html.A(html.Button('Download Data', id='download-button'), id='download-link-metasearch-1')
          ]),
        # Second Data Table
        html.Div([
            dash_table.DataTable(
              id='datatable-metasearch-2',
              columns=[{"name": i, "id": i} for i in df_columns_calculated],
              editable=True,
              n_fixed_columns=1,
              style_table={'maxWidth': '1500px'},
                  # sorting=True,
                  # sorting_type="multi",
                   style_cell = {"fontFamily": "Arial", "size": 10, 'textAlign': 'left'},
                   style_cell_conditional=[{'if': {'row_index': 'odd'}, 'backgroundColor': '#D5DBDB'}]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F4ECF7'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E8DAEF'} for c in ['CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)']]
                    + [{'if': {'column_id': c}, 'backgroundColor': '#FDEDEC'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#FADBD8'} for c in ['CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]]
                    + [{'if': {'column_id': c},  'backgroundColor': '#F6DDCC'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c, 'row_index': 'odd'}, 'backgroundColor': '#E59866'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', ]]
                    + [{'if': {'column_id': c}, 'minWidth': '0px', 'maxWidth': '80px', 'whiteSpace': 'normal'} for c in ['CPS - TY', 'CPS - LP', 'CPS PoP (Abs)', 'CPS - LY', 'CPS YoY (Abs)', 'CPS PoP (%)', 'CPS YoY (%)', 'CVR - TY', 'CVR - LP', 'CVR PoP (Abs)','CVR - LY', 'CVR YoY (Abs)', 'CVR PoP (%)', 'CVR YoY (%)', 'CPA - TY', 'CPA - LP', 'CPA PoP (Abs)', 'CPA - LY', 'CPA YoY (Abs)', 'CPA PoP (%)', 'CPA YoY (%)' ]],
            ),
            ], className=" twelve columns"),
        # GRAPHS
        html.Div([
            html.Div(
            id='update_graph_1'
            ),
            html.Div([
                dcc.Graph(id='metasearch'),
            ], className=" twelve columns"
            ),], className="row "
        ),
        ], className="subpage")
    ], className="page")

######################## END Metasearch and Travel Ads Layout ########################

######################## 404 Page ########################
noPage = html.Div([ 
    # CC Header
    Header(),
    html.P(["404 Page not found"])
    ], className="no-page")

ModuleNotFoundError: No module named 'components'